# Учебный проект 14_Прогнозирование риска ДТП

## Содержание

* [Описание проекта](#Описание)
* [Импорт библиотек Python](#Импорт)
* [Загрузка данных](#Загрузка)
* [Первичный анализ данных](#ПервичАнализ)
* [Блок машинного обучения](#Моделирование)
    * [Первичный отбор данных для построения модели](#Моделирование_отбор)
    * [Предобработка данных](#Моделирование_предобработка)
    * [Исследовательский анализ данных](#Моделирование_исследование)
        * [Анализ категориальных признаков](#Моделирование_исследование_категории)
        * [Анализ количественных признаков](#Моделирование_исследование_количество)
    * [Подготовка данных](#Моделирование_подготовка)
    * [Построение моделей прогнозирования риска ДТП](#Моделирование_baseline)
        * [Logistic Regression](#Моделирование_baseline_logistic)
        * [CatBoost](#Моделирование_baseline_catboost)
        * [Модель нейронной сети PyTorch](#Моделирование_baseline_pytorch)
    * [Подбор оптимальных гиперпараметров модели прогнозирования](#Моделирование_параметры)
    * [Анализ метрик моделей](#Моделирование_метрики)
    * [Проверка качества модели на тестовых данных](#Моделирование_качество)
    * [Анализ важности основных факторов](#Моделирование_факторы)
* [Общий вывод](#Вывод)

## Описание проекта <a class = 'anchor' id = 'Описание'></a>

На исследовании находятся данные о `поездках, совершенных через сервис каршеринга`.

---

`Задача`

Разработать и выбрать лучшую модель для **прогнозирования водительского риска по выбранному маршруту движения**.

---

`Идея решения задачи от заказчика`

1. Создать модель предсказания ДТП (целевое значение - at_fault (виновник) в таблице `parties`);
    * Для модели выбрать тип виновника - только машина (**car**);
    * Выбрать случаи, когда ДТП привело к любым повреждениям транспортного средства, кроме типа SCRATCH (царапина);
    * Для моделирования ограничиться данными за 2012 год - они самые свежие;
    * Обязательное условие - учесть фактор возраста автомобиля.
2. На основе модели исследовать основные факторы ДТП;
3. Понять, помогут ли результаты моделирования и анализ важности факторов ответить на вопросы:
    * Возможно ли создать адекватную систему оценки водительского риска при выдаче авто?
    * Какие еще факторы нужно учесть?
    * Нужно ли оборудовать автомобиль какими-либо датчиками или камерой?

`Путь решения задачи`

1. Собрать данные о поездках, водителях и характеристиках транспортных средств - выполнить запрос из базы данных;
2. Провести предобработку значений в наборах данных;
3. Провести исследовательский анализ данных для выявления закономерностей, применимых к последующей настройке моделей машинного обучения (МО);
4. Подготовить выборки для обучения моделей;
5. Построить baseline модели прогнозирования для получения предварительных результатов;
6. Подготовить пул моделей и их гиперпараметров и выбрать лучшее решение для прогнозирования водительского риска;
7. Сформировать вывод о подготовленных решениях.

---

`Располагаемые данные`

`collisions` - общая информация о ДТП

Имеет уникальный ключ **case_id**. Эта таблица описывает общую информацию о ДТП. Например, где оно произошло и когда.


`parties` - информация об участниках ДТП

Имеет неуникальный **case_id**, который сопоставляется с соответствующим ДТП в таблице `collisions`. Каждая строка здесь описывает одну из сторон, участвующих в ДТП. Если столкнулись две машины, в этой таблице должно быть две строки с совпадением **case_id**. Уникальный идентификатор - **case_id and party_number**


`vehicles` - информация о пострадавших машинах

Имеет неуникальные **case_id** и неуникальные **party_number**, которые сопоставляются с таблицей `collisions` и таблицей `parties`. Уникальный идентификатор - **case_id and party_number**.

## Импорт библиотек Python <a class = 'anchor' id = 'Импорт'></a>

1. Импорт библиотек Python для:
    * подключения к БД;
    * манипулирования данными;
    * визуализации данных;
    * проведения математических операций;
    * работы с датой и временем;
    * построения матрицы корреляции данных;
    * решения задач машинного обучения:
        * механизмы построения нейронных сетей;
        * метрики оценки эффективности моделей;
        * механизмы отбора данных;
        * механизмы подготовки данных.
2. Инициализация переменных-констант для последующего использования на этапе построения моделей МО;
3. Формирование вывода по итогам данного этапа.

In [1]:
# импорт библиотек python

# для подключения к базе данных
import psycopg2
import sqlalchemy
from psycopg2 import sql, extras

# для манипулирования данными
import pandas as pd
import numpy as np

# для визуализации данных
import matplotlib.pyplot as plt
import seaborn as sns

# установка размеров для последующих графиков в проекте
plt.rcParams['figure.figsize'] = (10, 5)

# библиотека математических операций
import math
# библиотека для работы с датой и временем
import datetime as dt

# вычисление корреляции данных
from phik import phik_matrix

# импорт библиотеки для построения нейронной сети
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from skorch.classifier import NeuralNetClassifier
from skorch.dataset import ValidSplit
from skorch.callbacks import EpochScoring, EarlyStopping

# метрики оценки эффективности моделей
from sklearn.metrics import mean_squared_error, accuracy_score, precision_score, recall_score, confusion_matrix

# механизмы отбора данных и подбора параметров моделей
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold, cross_val_score
from sklearn.compose import ColumnTransformer, make_column_transformer
import optuna
from optuna.samplers import TPESampler

# механизмы подготовки данных
from sklearn.preprocessing import (StandardScaler,
                                   OneHotEncoder,
                                   OrdinalEncoder)

# линейная модель классификации и механизмы формирования пайплайна
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# библиотека catboost
from catboost import CatBoostClassifier, Pool, cv

# анализ важности признаков
import shap

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# инициализация констант для дальнейшего использования в проекте
# инициализация переменной RANDOM_STATE для фиксирования "случайности"
RANDOM_STATE = 42
# инициализация переменной TEST_SIZE для разделения выборки на обучающую и тестовую
TEST_SIZE = 0.25
# количество сплитов набора данных для кросс-валидации
SPLITS = 4

In [ ]:
# данные для подключения к базе данных
db_config = {
'user': 'praktikum_student', # имя пользователя,
'pwd': 'Sdf4$2;d-d30pp', # пароль,
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения,
'db': 'data-science-vehicle-db' # название базы данных,
} 


**Вывод**

1. Выполнена установка необходимых библиотек Python:
    * phik - для построения матрицы корреляции;
    * optuna - поиск гиперпараметров для модели;
    * skorch - для возможности использования PyTorch совместно с Sklearn;
    * shap - для построения визуализации важности признаков;
    * выполнено обновление библиотеки scikit-learn.
2. Импортированы библиотеки Python:
    * для подключения к базе данных:
        * psycopg2;
        * sqlalchemy;
        * sql, extras.
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для проведения математических операций:
        * math
    * для работы с датой и временем:
        * datetime
    * для вычисления корреляции данных:
        * phik_matrix
    * для решения задач машинного обучения:
        * механизмы построения нейронной сети:
            * torch.nn.;
            * optim;
            * Dataset, DataLoader;
            * NeuralNetClassifier;
            * ValidSplit;
            * EpochScoring, EarlyStopping
        * метрики оценки эффективности моделей классификации;
        * train_test_split - механизм разделения данных;
        * StandardScaler - стандартизация данных;
        * OneHotEncoder - one-hot кодирование категориальных признаков;
        * OrdinalEncoder - кодирование значений ранговых признаков;
        * линейные модели классификации;
        * библиотека CatBoost;
        * библиотека shap для анализа важности признаков.
3. Инициализированы переменные **RANDOM_STATE**, **TEST_SIZE** и **SPLITS** для фиксирования "случайности", размера тестовой выборки и количества сплитов на этап кросс-валидации соответственно;
4. Инициализированы параметры подключения к БД.

## Загрузка данных <a class = 'anchor' id = 'Загрузка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Подключение к Базе Данных по установленным параметрам входа;
2. Инициализация пользовательской функции выполнения запросов - **request**. Выполнение тестового запроса;
3. Создание объекта **engine** для подключения к БД;
4. Вывод на экран основной информации о наборах данных;
5. Формирование вывода по итогам данного этапа.

In [ ]:
# установка соединения
try:
    connection = psycopg2.connect(
        user = db_config['user'],
        password = db_config['pwd'],
        host = db_config['host'],
        port = db_config['port'],
        dbname = db_config['db']
    )
    
    print("Успешное подключение к базе данных!")
    
    # инициализация объекта курсор для выполнения запросов
    cursor = connection.cursor()
    
    # выполнение простого запроса
    cursor.execute("SELECT version();")
    db_version = cursor.fetchone()
    print(f"Версия PostgreSQL: {db_version[0]}")
    
except Exception as error:
    print(f"Ошибка при подключении к базе данных: {error}")

In [ ]:
# функция для выполнения запросов
def request(query):
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        result_df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
        return result_df
    except Exception as e:
        print(f'Error: {e}')
        connection.rollback()  # откат транзакции, для избежания блокировки

In [ ]:
# выполнение тестового запроса
query = '''
SELECT case_id, county_city_location, county_location, distance, direction
FROM collisions
LIMIT 1;
'''
display(request(query))

In [ ]:
# создание объекта engine для подключения к БД
engine = sqlalchemy.create_engine(
    f"postgresql://{db_config['user']}:{db_config['pwd']}@{db_config['host']}:{db_config['port']}/{db_config['db']}"
)
# получение метаданных БД
metadata = sqlalchemy.MetaData()
metadata.reflect(bind=engine)

In [ ]:
# вывод на экран основной информации о наборах данных
for table_name, table in metadata.tables.items():
    query = f'SELECT COUNT(*) FROM {table_name};'
    print(f'''
Таблица: {table_name};
Количество записей: {request(query).iloc[0][0]};
Признаки: {table.columns.keys()};
Первичный ключ: {table.primary_key.columns.keys() if table.primary_key else 'Отсутствует'}     
           ''')

**Вывод**

1. Выполнено подключение к Базе Данных по установленным параметрам входа;
2. Проинициализирована пользовательская функция выполнения запросов - **request**. Выполнен тестовый запрос;
3. Инициализирован объект **engine** для подключения к БД;
4. Выведена на экран первичная информация о наборах данных:
    * `collisions`:
        * Общее количество записей - 1_400_000;
        * Общее количество признаков - 20;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `case_ids`:
        * Общее количество записей - 1_400_400;
        * Общее количество признаков - 2;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `parties`:
        * Общее количество записей - 2_752_408;
        * Общее количество признаков - 9;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `vehicles`:
        * Общее количество записей - 1_021_234;
        * Общее количество признаков - 6;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.

## Первичный анализ данных <a class = 'anchor' id = 'ПервичАнализ'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Проведение первичного анализа наборов данных:
    * изучение временного диапазона располагаемых данных;
    * построение сводных таблиц и визуализации для изучения взаимосвязей количества аварий и:
        * месяцев года;
        * частых причин нарушений;
        * погодных условий;
        * возраста транспортных средств / вида нарушения;
        * времени суток / состояния дороги;
2. Формирование вывод по итогам данного этапа.

In [ ]:
# анализ временных границ располагаемых данных
query = """SELECT MIN(collision_date) AS min_date,
                  MAX(collision_date) AS max_date
          FROM collisions;"""

display(request(query))

**Вывод по промежуточному этапу**

База данных имеет обширную информацию за 11 лет. На текущем этапе необходимо ограничить набор данных по 2019 год включительно (так как значения за 2020 год представлены не в полном виде).

In [ ]:
# инициализация пользовательской функции по формированию вывода информации
def display_info(df: pd.DataFrame, index_column : str, num_column: str, title: str, xlabel: str, kind_of_plot : str):
# построение визуализации по выбранной метрике
    if kind_of_plot == 'pie':
        (df
         .set_index(index_column)
         .sort_values(by = num_column, ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5), autopct='%1.0f%%'))
    else:
        (df
         .set_index(index_column)
         .sort_values(by = num_column, ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5)))
    ax = plt.gca()
    ax.axes.yaxis.set_visible(False)
    plt.xlabel(xlabel)
    plt.title(title, fontsize = 12)
    plt.show()

# построение сводной таблицы по выбранной метрике
    pivot_data = df
    pivot_data['share_of_collisions'] = round(pivot_data[num_column] / pivot_data[num_column].sum() * 100, 2)
    pivot_data.columns = [index_column, 'count_of_collisions', 'share_of_collisions']
    display(pivot_data)

In [ ]:
# анализ зависимости количества аварий от месяца
# выполняется фильтрация по году, так как данные за 2020 год не полные
query = """SELECT TO_CHAR(collision_date, 'Month') AS month_name,
                  COUNT(*) AS num_collisions
           FROM collisions
           WHERE EXTRACT(YEAR FROM collision_date) < 2020
           GROUP BY month_name
           ORDER BY num_collisions DESC;"""
# запись результата запроса в переменную
df = request(query)

# вывод информации на экран
display_info(df, 'month_name', 'num_collisions', 'Соотношение количества аварий от месяца года', 'Месяц года', 'bar')

**Вывод по промежуточному этапу**

* Топ-3 месяца по количеству аварий:
    * Март;
    * Январь;
    * Май.

Вероятно, это связано с погодными условиями (например, обильный снег и гололед в Январе; таяние снега в марте), так и с факторами праздничных дней и увеличением трафика на дорогах (обилие выходных дней в Мае, когда люди стремятся выехать за город; праздничные дни в Январе).

* Стабильность ситуации с ДТП в летне-осенний период. Вероятно, что причина этому благоприятные погодные условия и сухость дорожного полотна (летом);
* Сезонное увеличение количества ДТП. Летне-осенний период характеризуется меньшим числом аварий, чем зимне-весенний.

In [ ]:
# анализ зависимости количества аварий и причин дтп
query = """SELECT pcf_violation_category AS damage_factor,
                    COUNT(*) AS num_collisions
            FROM collisions
            WHERE EXTRACT(YEAR FROM collision_date) < 2020
            GROUP BY pcf_violation_category
            ORDER BY num_collisions DESC
            LIMIT 10;"""

# запись результата запроса в переменную
df = request(query)

# вывод информации на экран
display_info(df, 'damage_factor', 'num_collisions', 'Соотношение количества аварий от причин ДТП', 'Причина ДТП', 'bar')

**Вывод по промежуточному этапу**

* Топ-5 причин по количеству аварий:
    * Превышение скорости (speeding) - 34.5% от общего числа;
    * Неправильный поворот (improper turning) - 18.8%;
    * Приоритет движения автомобилей (automobile right of way) - 12.5%;
    * Езда в нетрезвом виде (dui) - 8.7%;
    * Опасное пересечение полосы движения (unsafe lane change) - 7.4%.

In [ ]:
# анализ зависимости количества аварий от погодных условий
query = """SELECT weather_1 AS weather_factor,
                    COUNT(*) AS num_collisions
            FROM collisions
            WHERE EXTRACT(YEAR FROM collision_date) < 2020
            GROUP BY weather_1
            ORDER BY num_collisions DESC;"""

# запись результата запроса в переменную
df = request(query)

# вывод информации на экран
display_info(df, 'weather_factor', 'num_collisions', 'Соотношение количества аварий от причин ДТП', 'Причина ДТП', 'bar')

**Вывод по промежуточному этапу**

* Топ-5 погодных условий по количеству аварий:
    * Ясно (clear) - 80.07% от общего числа;
    * Облачно (cloudy) - 14.5%;
    * Дождь (raining) - 4.1%;
    * Без указания погоды (None) - 0.5%;
    * Туман (fog) - 0.4%.

In [ ]:
# анализ зависимости количества аварий от возраста автомобиля и типа аварий
# новым считается автомобиль, возраст которого не более 3 лет
query = """SELECT 
                    CASE
                        WHEN vehicle_age <= 3 THEN 'Новые автомобили'
                        ELSE 'Старые автомобили'
                    END AS vehicle_age_category,
                    type_of_collision,
                    COUNT(*) AS num_collisions
            FROM collisions AS c
            JOIN vehicles AS v ON c.case_id = v.case_id
            WHERE vehicle_age IS NOT NULL AND type_of_collision IS NOT NULL AND EXTRACT(YEAR FROM collision_date) < 2020
            GROUP BY vehicle_age_category, type_of_collision
            ORDER BY vehicle_age_category ASC, num_collisions DESC;"""

# запись результата запроса в переменную
df = request(query)

# вывод информации на экран
display(df)

# визуализация результатов - написанная изначально функция не подходит для визуализации таких данных
plt.figure(figsize=(10, 6))
sns.barplot(x = 'type_of_collision', y = 'num_collisions', hue = 'vehicle_age_category', data=df)
plt.xlabel('Тип аварии')
plt.ylabel('Количество аварий')
plt.title('Распределение типов аварий в зависимости от возраста автомобиля')
plt.legend(title='Категория автомобиля', loc='upper right')
plt.xticks(rotation=45, ha='right')
plt.show()

**Вывод по промежуточному этапу**

* Независимо от возрастной категории автомобилей, самыми частыми типами аварий являются:
    * "rear end" (столкновение сзади);
    * "broadside" (боковое столкновение).
    
> Стоит отметить, что общее количество аварий с участием старых автомобилей выше, чем с участием новых. И это невзирая на сам тип аварии.

* Типы аварий остаются одинаковыми для двух возрастных категорий автомобилей (исключение составляют только типы sideswipe и hit object, которые "поменялись местами" для новых автомобилей)

In [ ]:
# анализ зависимости количества аварий от возраста автомобиля и типа аварий
# новым считается автомобиль, возраст которого не более 3 лет
query = """
SELECT 
CASE 
    WHEN EXTRACT(HOUR FROM collision_time) >= 0 AND EXTRACT(HOUR FROM collision_time) < 6 THEN 'Ночь'
    WHEN EXTRACT(HOUR FROM collision_time) >= 6 AND EXTRACT(HOUR FROM collision_time) < 12 THEN 'Утро'
    WHEN EXTRACT(HOUR FROM collision_time) >= 12 AND EXTRACT(HOUR FROM collision_time) < 18 THEN 'День'
    WHEN EXTRACT(HOUR FROM collision_time) >= 18 AND EXTRACT(HOUR FROM collision_time) < 24 THEN 'Вечер'
END AS time_of_day,
road_surface,
COUNT(*) AS collision_count
FROM collisions c
     JOIN vehicles v ON c.case_id = v.case_id
WHERE road_surface IS NOT NULL AND 
      collision_time IS NOT NULL AND
      EXTRACT(YEAR FROM COLLISION_DATE)<2020
GROUP BY time_of_day, road_surface
ORDER BY time_of_day, collision_count DESC;"""

# запись результата запроса в переменную
df = request(query)

# вывод информации на экран
display(df)

In [ ]:
# визуолизация результатов
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Анализ частоты ДТП в зависимости от времени суток и состояния дороги')

sns.barplot(x='time_of_day',
            y='collision_count',
            data=df[df['road_surface'] == 'dry'],
            ax=axes[0, 0],
            palette='Blues')
axes[0, 0].set_title('Сухая дорога')
axes[0, 0].set_xlabel('Время суток')
axes[0, 0].set_ylabel('Количество аварий')

sns.barplot(x='time_of_day',
            y='collision_count',
            data=df[df['road_surface'] == 'wet'],
            ax=axes[0, 1],
            palette='Greens')
axes[0, 1].set_title('Мокрая дорога')
axes[0, 1].set_xlabel('Время суток')
axes[0, 1].set_ylabel('Количество аварий')

sns.barplot(x='time_of_day', 
            y='collision_count', 
            data=df[df['road_surface'] == 'snowy'], 
            ax=axes[1, 0], 
            palette='Oranges')
axes[1, 0].set_title('Снежная дорога')
axes[1, 0].set_xlabel('Время суток')
axes[1, 0].set_ylabel('Количество аварий')

sns.barplot(x='time_of_day', 
            y='collision_count', 
            data=df[df['road_surface'] == 'slippery'], 
            ax=axes[1, 1], 
            palette='Reds')
axes[1, 1].set_title('Скользкая дорога')
axes[1, 1].set_xlabel('Время суток')
axes[1, 1].set_ylabel('Количество аварий')

plt.tight_layout()
plt.show()

**Вывод по промежуточному этапу**

* Наибольшее количество ДТП происходит на **сухой дороге**. Можно провести связь между подобными условиями для езды и одной из самых частых причин аварии - **превышение скорости**. Сухая дорога обеспечивает лучший контакт шин с дорогой, и невольно побуждает водителей ехать быстрее разрешенного ограничения скорости.


* С другой стороны, наличие дождя, снега или скользкого дорожного покрытия создает более опасные условия для движения, что заставляет водителей быть более осторожными, снизить скорость и быть более внимательными к дорожной обстановке или совсем отказаться от использования автомобиля. Это может приводить к снижению количества ДТП в условиях плохой погоды.


* Вне зависимости от состояния дороги (сухой, мокрый, снежный или скользкий), **большинство ДТП происходят утром и днем**. Это могут быть утренние рабочие часы, а также общий дневной трафик в рабочие и выходные дни


* В случае, когда **дорога заснежена** (снежное состояние), **наибольшее количество ДТП происходит именно утром**. Этому может быть объяснение в виде неочищенной дороги от наледи и снега.

**Вывод**

1. Проведен первичный анализ наборов данных:
    * Изучен временной диапазон располагаемых данных - **с 01.01.2009 по 26.07.2020**;
    * Построены сводные таблицы и визуализации для изучения взаимосвязей количества аварий и:
        * месяцев года:
            * Топ-3 месяца по количеству аварий:
                * Март;
                * Январь;
                * Май.
        * частых причин нарушений:
            * Превышение скорости (speeding) - 34.5% от общего числа;
            * Неправильный поворот (improper turning) - 18.8%;
            * Приоритет движения автомобилей (automobile right of way) - 12.5%;
            * Езда в нетрезвом виде (dui) - 8.7%;
            * Опасное пересечение полосы движения (unsafe lane change) - 7.4%.
        * погодных условий:
            * Ясно (clear) - 80.07% от общего числа;
            * Облачно (cloudy) - 14.5%;
            * Дождь (raining) - 4.1%;
            * Без указания погоды (None) - 0.5%;
            * Туман (fog) - 0.4%.
        * возраста транспортных средств / вида нарушения:
            * Независимо от возрастной категории автомобилей, самыми частыми типами аварий являются:
            * "rear end" (столкновение сзади);
            * "broadside" (боковое столкновение).
            * Типы аварий остаются одинаковыми для двух возрастных категорий автомобилей (исключение составляют только типы sideswipe и hit object, которые "поменялись местами" для новых автомобилей)
        * времени нарушения / состояния дороги:
            * Наибольшее количество ДТП происходит на **сухой дороге**. Можно провести связь между подобными условиями для езды и одной из самых частых причин аварии - **превышение скорости**. Сухая дорога обеспечивает лучший контакт шин с дорогой, и невольно побуждает водителей ехать быстрее разрешенного ограничения скорости.
            * С другой стороны, наличие дождя, снега или скользкого дорожного покрытия создает более опасные условия для движения, что заставляет водителей быть более осторожными, снизить скорость и быть более внимательными к дорожной обстановке или совсем отказаться от использования автомобиля. Это может приводить к снижению количества ДТП в условиях плохой погоды.
            * Вне зависимости от состояния дороги (сухой, мокрый, снежный или скользкий), **большинство ДТП происходят утром и днем**. Это могут быть утренние рабочие часы, а также общий дневной трафик в рабочие и выходные дни
            * В случае, когда **дорога заснежена** (снежное состояние), **наибольшее количество ДТП происходит именно утром**. Этому может быть объяснение в виде неочищенной дороги от наледи и снега.

##  Блок моделей машинного обучения <a class = 'anchor' id = 'Моделирование'></a>

###  Первичный отбор данных для построения модели <a class = 'anchor' id = 'Моделирование_отбор'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Первичный отбор данных для построения модели по условиям:
    * Виновник аварии - **car** (автомобиль);
    * Тип повреждений - значительные повреждения любого из участников (исключая **scratch** (царапина));
    * Временные рамки для моделирования - **2012 год**.
2. Отбор признаков для моделирования;
3. Формирование вывод по итогам данного этапа.

In [ ]:
# sql запрос
query = '''
SELECT *
FROM parties p INNER JOIN
     vehicles v ON p.case_id = v.case_id AND p.party_number = v.party_number LEFT JOIN
     collisions c ON p.case_id = c.case_id
WHERE p.case_id IN (SELECT case_id
                    FROM parties
                    WHERE PARTY_TYPE = 'car' AND
                          AT_FAULT = 1) AND
      c.COLLISION_DAMAGE NOT IN ('SCRATCH') AND
      EXTRACT(YEAR FROM c.COLLISION_DATE) = 2012;

'''

df = request(query)
display(df.head())

# закрываем курсор и соединение
cursor.close()
connection.close()

**Вывод по промежуточному этапу**

Признаки, которые не нужны для последующего построения модели МО:
* Показатели идентификации:
    * **id** - номер записи в БД;
    * **case_id** - идентификатор ДТП;
    * **party_number** - номер участника происшествия;
    * **county_city_location** - идентификатор географического района, где произошло ДТП.
* Показатели пост-анализа ДТП:
    * **party_count** - Количество участников
    * **primary_collision_factor** - Основной фактор аварии
    * **pcf_violation_category** - Категория нарушения
    * **type_of_collision** - Тип аварии
    * **motor_vehicle_involved_with** - Дополнительные участники ДТП
    * **intersection** - Является ли место происшествие перекрёстком
    * **collision_damage** - Серьёзность происшествия

In [ ]:
# фильтрация по типу участника происшествия - car
df = df[df['party_type'] == 'car']

# удаление ненужных признаков
df = df.drop(['id', 'case_id', 'party_number', 'party_type', 'county_city_location', 
             'party_count', 'primary_collision_factor', 'pcf_violation_category', 'type_of_collision', 
             'motor_vehicle_involved_with', 'intersection', 'collision_damage'], axis = 1)

**Вывод**

1. Произведен первичный отбор данных для построения модели по условиям:
    * Виновник аварии - **car** (автомобиль);
    * Тип повреждений - значительные повреждения любого из участников (исключая **scratch** (царапина));
    * Временные рамки для моделирования - **2012 год**.
2. Произведен отбор признаков для моделирования с исключением следующих показателей:
    * Показатели идентификации:
        * **id** - номер записи в БД;
        * **case_id** - идентификатор ДТП;
        * **party_number** - номер участника происшествия;
        * **county_city_location** - идентификатор географического района, где произошло ДТП.
    * Показатели пост-анализа ДТП:
        * **party_count** - Количество участников
        * **primary_collision_factor** - Основной фактор аварии
        * **pcf_violation_category** - Категория нарушения
        * **type_of_collision** - Тип аварии
        * **motor_vehicle_involved_with** - Дополнительные участники ДТП
        * **intersection** - Является ли место происшествие перекрёстком
        * **collision_damage** - Серьёзность происшествия 
3. Сформированный набор подготовлен для последующей предобработки данных и исследовательского анализа.

###  Предобработка данных <a class = 'anchor' id = 'Моделирование_предобработка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Вывод общей информации по сформированному для моделирования набору;
2. Проверка датасетов на явные дубликаты;
3. Формирование вывода по итогам данного этапа.

In [ ]:
# инициализация пользовательской функции для первичного изучения содержимого наборов данных
def first_meeting (df : pd.DataFrame, df_name : str) -> None:
    print(f'Структура набора данных {df_name}')
    display(df.head())
    print('Общая информация о наборе')
    print(df.info())
    print()

In [ ]:
# вывод на экран параметров датасета 'df'
first_meeting(df, 'df')

In [ ]:
# поиск количества и доли полных дубликатов
print('Количество полных дубликатов в наборе данных:', df.duplicated().sum())
print('Доля дубликатов от общего количества записей в наборе: {:.2%}'.format(df.duplicated().sum() / df.shape[0]))

In [ ]:
# исключение дублирующихся записей из набора данных
df = df.drop_duplicates()

**Вывод**

1. Выведена общая информация о наборе `df` на экран:
    * В наборе присутствуют пустые значения по нескольким признакам. Стратегия их заполнения будет выбрана на этапе исследовательского анализа данных;
    * Типы данных **соответствуют** сущностям значений по всем рассматриваемым признакам;
2. Выполнена проверка датасетов на явные дубликаты - **Дубликаты были удалены из набора**;
3. Набор данных подготовлен к дальнейшему исследовательскому анализу.

### Корреляционный анализ данных <a class = 'anchor' id = 'Моделирование_корреляция'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Построение матрицы корреляции - поиск признаков высокой взаимосвязи показателей объектов;
2. Проведение отбора признаков для последующего построения моделей машинного обучения;
3. Формирование вывода по итогам данного этапа.

In [ ]:
# построение матрицы корреляции по всем признакам с исключением дат
plt.figure(figsize=(15, 8))
(sns.heatmap(df
             .drop(['collision_date', 'collision_time'], axis = 1)
             .phik_matrix(interval_cols = ['insurance_premium', 'distance', 'vehicle_age']), annot = True))
plt.title('Phik-матрица корреляции между количественными и качественными показателями набора данных', fontsize = 12)
plt.show()

**Вывод по промежуточному этапу:**

При анализе матрицы корреляции по **количественным и качественным признакам** заметна линейная взаимосвязь между целевой переменной и указанными признаками:
* insurance_premium;
* party_sobriety;
* party_drug_physical;
* vehicle_transmission;
* county_location;
* road_surface;
* control_device

Слабую корреляцию с целевой переменной демонстрируют признаки:
* cellphone_in_use;
* vehicle_type;
* vehicle_age;
* distance;
* direction;
* weather_1;
* location_type;
* road_condition_1;
* lighting.

Наблюдается заметная корреляция между признаками:
* party_sobriety - party_drug_physical. Сильная взаимосвязь наблюдается по причине семантики самих признаков: Водитель под алкоголем является не трезвым, что дополнительно фиксируется в признаке party_drug_physical;
* road_surface - weather_1. Состояние дорожного покрытия зависит от погодных условий.

In [ ]:
# построение матрицы корреляции количественных показателей и поиск сильных взаимосвязей
plt.figure(figsize=(15, 8))
sns.heatmap(df.select_dtypes(include = 'number').corr(method = 'spearman'), annot = True) # используется построение корреляции по Спирману, так как количественные значения не распределены нормально
plt.title('Матрица корреляции Спирмана между количественными показателями набора данных', fontsize = 12)
plt.show()

**Вывод по промежуточному этапу:**

При анализе матрицы корреляции по **количественным признакам с применением коэффициента Спирмана** заметна линейная взаимосвязь между целевой переменной и указанными признаками. Стоит отметить, что наблюдается именно обратная линейная взаимосвязь:
* insurance_premium. Чем выше страховка, тем меньше вероятность возникновения ДТП. Такая связь объясняется дороговизной самого автомобиля с сопутствующей на него суммой страховки;
* vehicle_age. Чем старше автомобиль, тем меньше вероятность возникновения ДТП. Возможно, что это объясняется тем, что водители подобных транспортных средств обладают большим опытом вождения;

**Вывод**

1. Построена Phik-матрица корреляции - произведен анализ взаимосвязей **количественных и качественных** признаков с целевой переменной. Заметна линейная взаимосвязь между целевой переменной и указанными признаками:
* **insurance_premium**;
* **party_sobriety**;
* **party_drug_physical**;
* **vehicle_transmission**;
* **county_location**;
* **road_surface**;
* **control_device**

Слабую корреляцию с целевой переменной демонстрируют признаки:
* **cellphone_in_use**;
* **vehicle_type**;
* **vehicle_age**;
* **distance**;
* **direction**;
* **weather_1**;
* **location_type**;
* **road_condition_1**;
* **lighting**.

Наблюдается заметная корреляция между признаками:
* **party_sobriety - party_drug_physical**. Сильная взаимосвязь наблюдается по причине семантики самих признаков: Водитель под алкоголем является не трезвым, что дополнительно фиксируется в признаке party_drug_physical;
* **road_surface - weather_1**. Состояние дорожного покрытия зависит от погодных условий.

2. Построена матрица корреляции Спирмана - произведен анализ взаимосвязей количественных признаков с целевой переменной:
* **insurance_premium**. Чем выше страховка, тем меньше вероятность возникновения ДТП. Такая связь объясняется дороговизной самого автомобиля с сопутствующей на него суммой страховки;
* **vehicle_age**. Чем старше автомобиль, тем меньше вероятность возникновения ДТП. Возможно, что это объясняется тем, что водители подобных транспортных средств обладают большим опытом вождения.

Для этапа построения моделей машинного обучения будут отброшены следующие показатели, но после проведения исследовательского анализа данных и разбора семантики значений:
* **cellphone_in_use**;
* **vehicle_type**;
* **distance**;
* **direction**;
* **location_type**;
* **road_condition**

###  Исследовательский анализ данных <a class = 'anchor' id = 'Моделирование_исследование'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Вывод общей статистической информации по набору данных;
2. Инициализация пользовательских функций отображения информации по категориальным и количественным показателям;
3. Анализ категориальных признаков:
    * at_fault - виновность в ДТП;
    * cellphone_in_use - наличие телефона в автомобиле;
    * vehicle_age - возраст автомобиля (в годах);
    * party_sobriety - трезвость участника ДТП;
    * party_drug_physical - состояние участника: физическое или с учётом принятых лекарств;
    * vehicle_type - тип кузова;
    * vehicle_transmission - тип КПП;
    * county_location - названия географических районов;
    * direction - направление движения;
    * weather_1 - погодные условия;
    * location_type - тип дороги;
    * road_surface - состояние дороги;
    * lighting - освещение;
    * control_device - работоспособность устройств управления;
    * collision_time - время происшествия;
    * collision_date - дата происшествия;
    * road_condition_1 - дорожное состояние.
4. Анализ количественных признаков:
    * distance - расстояние до главной дороги;
    * vehicle_age - возраст автомобиля (в годах);
    * insurance_premium - сумма страховки.
5. Формирование вывода по итогам данного этапа.

In [ ]:
# вывод на экран основных статистик по набору 'df'
print(f'Основная статистическая информация по набору df')
df.describe().T

**Вывод по промежуточному этапу**

Набор `df` характеризуется следующими статистическими показателями:

* **at_fault - виновность участника**:
    * Среднее значение - 0.48;
    * Минимальное значение - 0.0;
    * Максимальное значение - 1.0;
    * Стандартное отклонение - 0.5.
* **insurance_premium - сумма страховки**:
    * Среднее значение - 38.5;
    * Минимальное значение - 0.0;
    * Максимальное значение - 104.0;
    * Стандартное отклонение - 16.2.
* **cellphone_in_use - наличие телефона в машине**:
    * Среднее значение - 0.02;
    * Минимальное значение - 0.0;
    * Максимальное значение - 1.0;
    * Стандартное отклонение - 0.14.
* **vehicle_age - возраст автомобиля**:
    * Среднее значение - 5.2;
    * Минимальное значение - 0.0;
    * Максимальное значение - 161.0;
    * Стандартное отклонение - 3.1.
* **distance - расстояние до главной дороги**:
    * Среднее значение - 701.8;
    * Минимальное значение - 0.0;
    * Максимальное значение - 1_584_000.0;
    * Стандартное отклонение - 5 669.3.

In [ ]:
# обновление пользовательской функции по формированию вывода информации

def display_info(df: pd.DataFrame, column_name: str, title: str, xlabel: str, kind_of_plot : str):
# построение визуализации по выбранной метрике
    plt.title(title, fontsize = 12)
    if kind_of_plot == 'pie':
        (df[column_name]
         .value_counts()
         .sort_values(ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5), autopct='%1.0f%%'))
    else:
        (df[column_name]
         .value_counts()
         .sort_values(ascending=True)
         .plot(kind = kind_of_plot, figsize = (8, 5)))
    ax = plt.gca()
    ax.axes.yaxis.set_visible(False)
    plt.xlabel(xlabel)
    plt.show()

# построение сводной таблицы по выбранной метрике
    pivot_data = (df[column_name]
                  .value_counts()
                  .sort_values(ascending=False)
                  .to_frame())
    pivot_data['share_of_values'] = round(pivot_data[column_name] / pivot_data[column_name].sum() * 100, 2)
    pivot_data.columns = ['count_of_values', 'share_of_values']
    display(pivot_data)

In [ ]:
# инициализация пользовательской функции по построению гистограмм по передаваемым метрикам
def histogram_plotting(data: pd.DataFrame, feature : str, bins: int, x_size: int, y_size: int, feature_xlabel : str):
    # вычисление статистических метрик для дальнейшей визуализации
    q1 = data[feature].quantile(0.25)
    q3 = data[feature].quantile(0.75)
    upper_bound = q3 + 1.5 * (q3 - q1)
    lower_bound = q1 - 1.5 * (q3 - q1)

    # построение визуализации
    plt.figure(figsize = (x_size, y_size))
    plt.hist(data[feature], color = 'blue', edgecolor = 'white', bins = bins)
    plt.axvline(upper_bound, c = 'red', ls = '-', label = 'верхняя граница допустимых значений')
    plt.axvline(q3, c = 'red', ls = '--', label = '3 квартиль значений')
    plt.axvline(q1, c = 'black', ls = '--', label = '1 квартиль значений')
    plt.axvline(lower_bound, c = 'black', ls = '-', label = 'нижняя граница допустимых значений')
    plt.title(f'Гистограмма распределения значений по метрике: {feature_xlabel}', fontsize = 10)
    plt.xlabel(feature_xlabel)
    plt.ylabel('Количество значений по метрике')
    plt.legend(bbox_to_anchor = (1, 0.6))
    plt.show()

    # вывод статистических метрик на экран
    print('Верхняя допустимая граница значений:', upper_bound)
    print('Нижняя допустимая граница значений:', lower_bound)
    print('Медианное значение:', data[feature].median())
    print('Среднее значение:', round(data[feature].mean(), 2))

    # расчет доли аномальных значений по метрике
    print('Доля значений, выходящих за верхнюю границу: {:.2%}'.format(data[data[feature] > upper_bound].shape[0] / data[feature].shape[0]))
    print('Доля значений, выходящих за нижнюю границу: {:.2%}'.format(data[data[feature] < lower_bound].shape[0] / data[feature].shape[0]))

####  Анализ категориальных признаков <a class = 'anchor' id = 'Моделирование_исследование_категории'></a>

##### Признак 'at_fault'

In [ ]:
# количество пропусков в целевом признаке - at_fault
print('Количество пропусков в целевом признаке:', df.at_fault.isna().sum())

# вывод информации по целевому признаку
display_info(df, 'at_fault', 'Соотношение записей по виновникам ДТП', 'Виновник в ДТП', 'pie')

**Вывод по промежуточному этапу**

* Пропуски по целевой переменной отсутствуют;
* Баланс целевой переменной хороший: 52% на 48%.

##### Признак 'cellphone_in_use'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.cellphone_in_use.isna().sum())

# вывод информации по признаку - cellphone_in_use (наличие телефона в машине)
display_info(df, 'cellphone_in_use', 'Соотношение записей по наличию телефона в машине', 'Наличие телефона', 'pie')

**Вывод по промежуточному этапу**

* На этапе корреляционного анализа была выявлена слабая связь данного признака с целевой переменной;
* В настоящее время водители для связи используют либо персональные смартфоны, либо встроенные средства связи в мультимедию автомобиля.

По совокупности данных причин признак можно исключить из набора данных для дальнейшего использования в построении моделей машинного обучения.

In [ ]:
# исключение признакак 'cellphone_in_use' из набора данных
df = df.drop('cellphone_in_use', axis = 1)

##### Признак 'party_sobriety'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.party_sobriety.isna().sum())

# вывод информации по признаку - party_sobriety (трезвость участника ДТП)
display_info(df, 'party_sobriety', 'Соотношение записей по трезвости участника ДТП', 'Трезвость участника', 'bar')

**Вывод по промежуточному этапу**

* Подавляющее большинство участников ДТП **не употребляли алкоголь на момент ДТП** - 91%;
* Заметна доля **употреблявших алкоголь на момент ДТП** - 5.7%;

Данный признак содержит несколько градаций трезвости:
* A — Had Not Been Drinking (Не пил)
* B — Had Been Drinking, Under Influence (Был пьян, под влиянием)
* C — Had Been Drinking, Not Under Influence (Был пьян, не под влиянием)
* D — Had Been Drinking, Impairment Unknown (Был пьян, ухудшение неизвестно)
* G — Impairment Unknown (Неизвестно ухудшение)
* H — Not Applicable (Не оценивался)
* Nan - Not Stated (Не указано)

Так как многие значения похожи между собой, можно создать новые группы, куда будут объединены похожие значения
* 0 - Had Not Been Drinking, Impairment Unknown, Not Applicable, Not Stated
* 1 - Had Been Drinking, Not Under Influence
* 2 - Had Been Drinking, Impairment Unknown
* 3 - Had Been Drinking, Under Influence

In [ ]:
# заполнение пропусков по признаку
df['party_sobriety'] = df['party_sobriety'].fillna(0)

# инициализация пользовательской функции по группировке значений
def replace_sobriety(x):
    if x in ['had not been drinking', 'impairment unknown', 'not applicable']: return 0
    elif x == 'had been drinking, not under influence': return 1
    elif x == 'had been drinking, impairment unknown' : return 2
    else: return 3

# замена значений по признаку
df['party_sobriety'] = df['party_sobriety'].apply(replace_sobriety)

In [ ]:
# проверка результата - вывод на экран списка уникальных значений
print('Список уникальных значений по признаку party_sobriety:', df.party_sobriety.unique())

# вывод на экран фрейма с подсчетом количества значений
party_sob_frame = df.party_sobriety.value_counts().to_frame().rename(columns = {'party_sobriety' : 'total_values'})
party_sob_frame['share_of_values'] = round(party_sob_frame['total_values'] * 100.0 / party_sob_frame['total_values'].sum(), 2)
display(party_sob_frame)

**Вывод по промежуточному этапу**

1. Выполнено заполнение пропусков - для всех случаев указано **значение 0**;
2. Выполнено создание групп значений - **похожие по семантике значения помещены в одну группу**. Группам присвоены значение от 0 до 3 включительно;
3. Подавляющее большниство водителей либо **не были пьяны, либо факт опьянение не был диагностирован** - 91.8% от общего количества записей.

##### Признак 'party_drug_physical'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.party_drug_physical.isna().sum())

# вывод информации по признаку - party_sobriety (трезвость участника ДТП)
display_info(df, 'party_drug_physical', 'Соотношение записей по состоянию участника ДТП', 'Состояние участника ДТП', 'bar')

In [ ]:
# заполнение пропусков по признаку
df['party_sobriety'] = df['party_sobriety'].fillna(0)

**Вывод по промежуточному этапу**

vebnterber

In [ ]:
# инициализация пользовательской функции по группировке значений
def replace_drug_physical(x):
    if x in ['under drug influence', 'sleepy/fatigued', 'impairment - physical']: return 1
    else: return 0

# замена значений по признаку
df['party_drug_physical'] = df['party_drug_physical'].apply(replace_drug_physical)

In [ ]:
# проверка результата - вывод на экран списка уникальных значений
print('Список уникальных значений по признаку party_sobriety:', df.party_drug_physical.unique())

# вывод на экран фрейма с подсчетом количества значений
party_drug_frame = df.party_drug_physical.value_counts().to_frame().rename(columns = {'party_drug_physical' : 'total_values'})
party_drug_frame['share_of_values'] = round(party_drug_frame['total_values'] * 100.0 / party_drug_frame['total_values'].sum(), 2)
display(party_drug_frame)

##### Признак 'vehicle_type'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.vehicle_type.isna().sum())

# вывод информации по признаку - vehicle_type (тип кузова транспортного средства)
display_info(df, 'vehicle_type', 'Соотношение записей по типу кузова ТС', 'Тип кузова', 'pie')

**Вывод по промежуточному этапу**

* Наблюдается слабая корреляция данного признака с целевой переменной;
* Семантика данного признака слабо репрезентативная - наличие того или иного типа кузова не может являться причиной ДТП.

По совокупности причин принимается исключить данный признак из набора данных.

In [ ]:
# исключение признака 'vehicle_type'
df = df.drop('vehicle_type', axis = 1)

##### Признак 'vehicle_transmission'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.vehicle_transmission.isna().sum())

# вывод информации по признаку - vehicle_transmission (тип коробки передач автомобиля)
display_info(df, 'vehicle_transmission', 'Соотношение записей по типу коробки передач', 'Тип КПП', 'pie')

**Вывод по промежуточному этапу**

* Наблюдается заметная корреляция данного признака с целевой переменной;
* Признак репрезентативен - наличие автоматической коробки передач уменьшает количество дополнительных действий и повышает скорость реакции при возникновении аварийной ситуации.

По совокупности причин принимается оставить данный признак в наборе данных.

Однако восстановить пропущенные значения не удастся за неимением информации. Необходимо удалить строки с пропущенными значениями из набора данных.

In [ ]:
# исключение строк с пустыми значениями по признаку
df = df.dropna(subset = ['vehicle_transmission'])

##### Признак 'county_location'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.county_location.isna().sum())

# вывод информации по признаку - county_location (названия географических районов ДТП)
display_info(df, 'county_location', 'Соотношение записей по названию географического района', 'Названия географических районов', 'bar')

**Вывод по промежуточному этапу**

Пустых значений по признаку не было выявлено.

На этапе корреляционного анализа была обнаружена заметная корреляция между признаком и целевой переменной. Однако количество уникальных названий локаций слишком велико, и работа со значениями (в том числе и на этапе обучения моделей) может быть затруднена.

**Необходимо сгруппировать значения по агломерациям**

In [ ]:
# инициализация словаря агломераций
agglomerations = {
    'Southern California': ['los angeles', 'ventura', 'orange', 'riverside', 'san bernardino', 'imperial'],
    'San Francisco Bay Area': ['san francisco', 'santa clara', 'alameda', 'contra costa', 'san mateo', 'marin'],
    'Central Valley': ['san joaquin', 'merced', 'tulare', 'fresno', 'kern', 'stanislaus'],
    'Sacramento Metropolitan Area': ['sacramento', 'placer', 'el dorado'],
    'Central Coast': ['santa cruz', 'monterey', 'san luis obispo', 'santa barbara', 'ventura', 'san benito', 'kings'],
    'North Coast': ['napa', 'sonoma', 'mendocino', 'humboldt'],
    'Inland Empire': ['riverside', 'san bernardino'],
    'San Diego Metropolitan Area': ['san diego'],
    'Sierra Nevada': ['nevada', 'calaveras', 'madera', 'tuolumne', 'amador', 'tehama', 'glenn', 'mariposa', 'sierra', 'plumas', 'trinity', 'lassen', 'plumas', 'trinity', 'sierra', 'modoc'],
    'Northern California': ['butte', 'sutter', 'yolo', 'lake', 'yuba', 'yolo', 'inyo', 'mono', 'yuba', 'shasta', 'solano'],
    'Far Northern California': ['del norte', 'humoldt', 'siskiyou', 'alpine', 'colusa']
}

# инициализация пользовательской функции замены значений
def county_to_agglomeration(county):
    for agglomeration, counties in agglomerations.items():
        if county.lower() in [c.lower() for c in counties]:
            return agglomeration
    return county

# замена значений по признаку 'county_location' и формирование групп
df['county_location'] = df['county_location'].map(county_to_agglomeration)

In [ ]:
# ПОВТОРНЫЙ вывод информации по признаку - county_location (названия географических районов ДТП)
display_info(df, 'county_location', 'Соотношение записей по названию географического района', 'Названия географических районов', 'bar')

**Вывод по промежуточному этапу**

Топ-5 групп районов по количеству совершенных ДТП:
* Southern California - 54.6%;
* San Francisco Bay Area - 12.4%;
* Central Valley - 8.7%;
* San Diego Metropolitan Area - 7.4%;
* Sacramento Metropolitan Area - 5.5%

##### Признак 'direction'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.direction.isna().sum())

# вывод информации по признаку - direction (направление движения)
display_info(df, 'direction', 'Соотношение записей по направлению движения', 'Названия направлений', 'pie')

**Вывод по промежуточному этапу**

На диаграмме видно равномерное распределение количества значений по разным направлениям.

Однако связь с целевой переменной слишком слабая - направление движение не репрезентативно по отношению к факту возникновения ДТП.

Можно исключить данный признак из набора данных.

In [ ]:
# исключение признака из набора данных
df = df.drop('direction', axis = 1)

##### Признак 'weather_1'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.weather_1.isna().sum())

# вывод информации по признаку - weather_1 (погодные условия)
display_info(df, 'weather_1', 'Соотношение записей по погодным условиям', 'Состояние погоды', 'bar')

**Вывод по промежуточному этапу**

Зафиксировано наличие пропущенных значений - 241 (возможно не было причин явно указывать состояние погоды по причине отсутствия плохих условий).

Подавляющее большинство ДТП происходит в ясную погоду. Можно создать бинарный признак - разделить погодные условия на 0 (ясная погода и пустые значения) и 1 (все другие погодные условия).

In [ ]:
# заполнение пустых значений по признаку
df['weather_1'] = df['weather_1'].fillna(0)

# замена значений погодных условий согласно поставленному условию
df['weather_1'] = df['weather_1'].apply(lambda x: 0 if x == 'clear' else 1)

In [ ]:
# ПОВТОРНЫЙ вывод информации по признаку - weather_1 (погодные условия)
display_info(df, 'weather_1', 'Соотношение записей по погодным условиям', 'Состояние погоды', 'bar')

##### Признак 'location_type'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.location_type.isna().sum())

# вывод информации по признаку - location_type (тип дороги)
display_info(df, 'location_type', 'Соотношение записей по типу дороги', 'Тип дороги', 'bar')

**Вывод по промежуточному этапу**

На этапе корреляционного анализа данных была выявлена слабая корреляция признака с целевой переменной.

Также наблюдается большое количество пустых значений, заполнение которых невозможно из-за недостатка информации и контекста о задаче.

Можно исключить данных признак из набора данных.

In [ ]:
# исключение признака из набора
df = df.drop('location_type', axis = 1)

##### Признак 'road_surface'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.road_surface.isna().sum())

# вывод информации по признаку - road_surface (состояние дороги)
display_info(df, 'road_surface', 'Соотношение записей по состоянию дороги', 'Состояние дороги', 'bar')

**Вывод по промежуточному этапу**

Данный признак показывает заметную корреляцию с целевой переменной.

Можно выполнить бинарное кодирование значений по признаку - разделить значения на 0 (дорога сухая или состояние явно не обозначено - пустые значения) и 1 (зафиксированы другие состояния дороги).

In [ ]:
# заполнение пустот по признаку
df['road_surface'] = df['road_surface'].fillna(0)

# замена значений по признаку на бинарные показатели
df['road_surface'] = df['road_surface'].apply(lambda x: 0 if x == 'dry' else 1)

In [ ]:
# ПОВТОРНЫЙ вывод информации по признаку - road_surface (состояние дороги)
display_info(df, 'road_surface', 'Соотношение записей по состоянию дороги', 'Состояние дороги', 'bar')

##### Признак 'lighting'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.lighting.isna().sum())

# вывод информации по признаку - lighting (освещение дороги)
display_info(df, 'lighting', 'Соотношение записей по типу освещения дороги', 'Тип освещения', 'bar')

**Вывод по промежуточному этапу**

Данный признак репрезентативную силу, но при этом слабую корреляцию - можно оставить его в наборе данных.

Можно выполнить бинарное кодирование значений по признаку - разделить значения на 0 (дорога освещена и у водителя есть видимость окружающей обстановки, а также для пустых значений) и 1 (присутствуют факторы, затрудняющие обзор).

In [ ]:
# заполнение пустот по признаку
df['lighting'] = df['lighting'].fillna(0)

# замена значений по признаку на бинарные показатели
df['lighting'] = df['lighting'].apply(lambda x: 0 if x in ['daylight', 'dark with street lights'] else 1)

In [ ]:
# ПОВТОРНЫЙ вывод информации по признаку - lighting (освещение дороги)
display_info(df, 'lighting', 'Соотношение записей по типу освещения дороги', 'Тип освещения', 'bar')

##### Признак 'control_device'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.control_device.isna().sum())

# вывод информации по признаку - control_device (работоспособность устройств управления)
display_info(df, 'control_device', 'Соотношение записей по работоспособности устройств', 'Устройство управления', 'bar')

**Вывод по промежуточному этапу**

Обнаружены пустые значения в количестве 411 шт.

Так как в описании к таблицам в базе данных нет точной расшифровки значений по данному признаку, то показатель **none** можно трактовать как:
* Устройство управления **отсутствует**;
* Нет информации о устройстве управления в транспортном средстве.

Предлагается заменить пустые значения и значения **none** на **no_info**.

In [ ]:
# заполнение пустот по признаку
df['control_device'] = df['control_device'].fillna('no_info')

# замена значений
df['control_device'] = df['control_device'].apply(lambda x: 'no_info' if x=='none' else x)

In [ ]:
# ПОВТОРНЫЙ вывод информации по признаку - control_device (работоспособность устройств управления)
display_info(df, 'control_device', 'Соотношение записей по значениям устройств управления', 'Работоспособность УУ', 'bar')

##### Признак 'collision_time'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.collision_time.isna().sum())
print('Доля пропущенных значений по признаку: {:.2%}'.format(df.collision_time.isna().sum() / df.shape[0]))

**Вывод по промежуточному этапу**

Восстановить данные о времени, восстановить или найти логичную замену по значениям нет возможносни - **будет логично их удалить**.

In [ ]:
# исключение строк с пропущенными значениями по признаку
df = df.dropna(subset = ['collision_time'])

**Вывод по промежуточному этапу**

Работа с текущими значениями времени не имеет смысла - необходимо преобразовать значения времени в категории по времени суток:
* Утро;
* День;
* Вечер;
* Ночь.

In [ ]:
# преобразование столбца 'collision_time' в формат времени
df['collision_time'] = pd.to_datetime(df['collision_time'], format='%H:%M:%S')

# округление часов до ближайшего значения
df['collision_time'] = df['collision_time'].dt.round('H').dt.hour


# инициализация пользовательской функции категоризации значений времени
def get_time_category(hour):
    if 0 <= hour < 6:
        return 'Ночь'
    elif 6 <= hour < 12:
        return 'Утро'
    elif 12 <= hour < 18:
        return 'День'
    else:
        return 'Вечер'

# преобразование значений часов в категории времени суток
df['collision_time'] = df['collision_time'].apply(get_time_category)

##### Признак 'collision_date'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.collision_date.isna().sum())
print('Доля пропущенных значений по признаку: {:.2%}'.format(df.collision_date.isna().sum() / df.shape[0]))

In [ ]:
# преобразование столбца 'collision_date' в формат даты
df['collision_date'] = pd.to_datetime(df['collision_date'])

# извлечение номера месяца
df['collision_date'] = df['collision_date'].dt.month

**Вывод по промежуточному этапу**

Значения месяца в текущем их виде не репрезентативны - необходимо преобразовать значения к группам времени года.

In [ ]:
# инициализация пользовательской функции преобразования значений номера месяца к времени года
def get_season(month):
    if month in [12, 1, 2]:
        return 'Зима'
    elif month in [3, 4, 5]:
        return 'Весна'
    elif month in [6, 7, 8]:
        return 'Лето'
    else:
        return 'Осень'
    
# преобразование значений по признаку
df['collision_date'] = df['collision_date'].apply(get_season)

##### Признак 'road_condition_1'

**Вывод по промежуточному этапу**

Так как на этапе корреляционного анализа данных была обнаружена слабая взаимосвязь между признаком и целевой переменной, можно исключить его из набора данных.

In [ ]:
# исключение признака из набора
df = df.drop('road_condition_1', axis = 1)

####  Анализ количественных признаков <a class = 'anchor' id = 'Моделирование_исследование_количество'></a>

##### Признак 'vehicle_age'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.vehicle_age.isna().sum())

# вывод информации по признаку - vehicle_age (возраст ТС)
display_info(df, 'vehicle_age', 'Соотношение записей в зависимости от возраста автомобиля', 'Возраст автомобиля', 'bar')

**Вывод по промежуточному этапу**

* Количество пропусков по признаку - 2 421 записей. Восстановить пропущенные значения не удастся, поэтому проще будет их исключить из общего набора данных

In [ ]:
# обработка пропусков
df = df.dropna(subset = ['vehicle_age'])

In [ ]:
# построение  гистограммы распределения значений по признаку 'vehicle_age'
histogram_plotting(df, 'vehicle_age', 100, 10, 5, 'Возраст автомобилей')

**Вывод по промежуточному этапу**

Стоит исключить аномально большие значения из набора данных по максимально допустимой границе.

In [ ]:
# исключение аномально больших значений из набора данных по признаку 'vehicle_age'
count_old = df.shape[0]
df = df[df['vehicle_age'] <= 13]
count_new = df.shape[0]

print('Доля исключенных аномальных значений от исходного количества строк: {:.1%}'.format((count_old - count_new) / count_old))

##### Признак 'distance'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.distance.isna().sum())

# построение  гистограммы распределения значений по признаку 'distance'
histogram_plotting(df, 'distance', 50, 10, 5, 'Расстояние до главной дороги')

**Вывод по промежуточному этапу**

На этапе корреляционного анализа была выявлена слабая связь данного признака с целевой переменной. Этот признак можно исключить из набора для дальнейшего использования в машинном обучении.

In [ ]:
# исключение признака 'distance' из набора данных
df = df.drop('distance', axis = 1)

##### Признак 'insurance_premium'

In [ ]:
# анализ пропусков по признаку
print('Количество пропущенных значений:', df.insurance_premium.isna().sum())

# построение  гистограммы распределения значений по признаку 'insurance_premium'
histogram_plotting(df, 'insurance_premium', 50, 10, 5, 'Сумма страховки')

**Вывод по промежуточному этапу**

* Количество пропусков по признаку - 752 записи. В данном случае можно заполнить пропуски значением "0" - это означает, что у человека нет страховки, и он за нее не платит;
* Доля аномально больших значений - 0.27%

In [ ]:
# заполнение пропусков значением "0"
df['insurance_premium'] = df['insurance_premium'].fillna(0)

#### Итоговый набор данных

In [ ]:
# просмотр текущего набора данных
first_meeting(df, 'df')

### Подготовка данных <a class = 'anchor' id = 'Моделирование_подготовка'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Отбор категориальных и количественных значений в отдельные переменные - инициализация переменных **categorial_columns** и **numeric_columns**;
2. Подготовка набора данных:
    * Разделение исходного набора `df` на область признаков и вектор целевой переменной - инициализация переменных `X` и **y** соответственно;
    * Инициализация переменных `X_train`, `X_test`, **y_train** и **y_test** для хранения обучающих и тестовых наборов данных;
3. Проверка баланса классов в наборах данных;
4. Формирование вывода по итогам данного этапа.

In [ ]:
# инициализация переменной для хранения категориальных признаков
categorical_columns = ['party_drug_physical', 'vehicle_transmission', 'county_location',
                       'weather_1', 'road_surface', 'lighting', 'collision_time',
                       'collision_date', 'control_device']

# инициализация переменной для хранения количественных признаков
numeric_columns = ['insurance_premium', 'vehicle_age', 'party_sobriety']


# инициализация переменной для хранения области признаков
X = df[categorical_columns + numeric_columns]
# инициализация переменной для хранения вектора целевой переменной
y = df['at_fault']

In [ ]:
# инициализация переменных для хранения обучающих и тестовых наборов данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = TEST_SIZE, stratify = y, random_state = RANDOM_STATE)

In [ ]:
# проверка баланса классов в выборках
print(f'''Баланс классов целевого признака:
y_train:
{round(y_train.value_counts()/len(y_train)*100)}
y_test:
{round(y_test.value_counts()/len(y_test)*100)}
''')

**Вывод**

1. Произведен отбор категориальных и количественных значений в отдельные переменные - **categorial_columns** и **numeric_columns**;
2. Подготовлены наборы данных для обучения и тестирования моделей:
    * Разделение исходного набора `df` на область признаков и вектор целевой переменной - инициализация переменных `X` и **y** соответственно;
    * Инициализация переменных `X_train`, `X_test`, **y_train** и **y_test** для хранения обучающих и тестовых наборов данных;
3. Проведена проверка баланса классов в наборах данных. Баланс классов в выборках сохранен.

### Построение моделей прогнозирования риска ДТП <a class = 'anchor' id = 'Моделирование_baseline'></a>

#### Logistic Regression <a class = 'anchor' id = 'Моделирование_baseline_logistic'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Преобразование входных данных для моделей - кодирование категориальных признаков и масштабирование количественных показателей;
2. Инициализация переменной **scorer** для хранения списка метрик качества оценки модели;
3. Инициализация объекта кросс-валидации с указанием параметров - **cv**;
4. Построение пайплайна преобразования данных и обучения модели логистической регрессии;
5. Обучение модели с перебором значений гиперпараметров;
6. Вывод на экран результатов обучения модели логистической регрессии;
7. Формирование вывода по итогам данного этапа.

In [ ]:
# преобразование входных данных
col_transformer_ohe = make_column_transformer(
    (
        OneHotEncoder(drop = 'first', handle_unknown = 'ignore'),
        categorical_columns
    ),
    (
        StandardScaler(),
        numeric_columns
    ),
    remainder = 'passthrough',
    verbose_feature_names_out = False
)

# инициализация списка метрик качества
scorer = ['accuracy', 'precision', 'recall']

# инициализация объекта кросс-валидации
cv = StratifiedKFold(n_splits = SPLITS, shuffle = True, random_state = RANDOM_STATE)

# инициализация объекта логистической регрессии
log_reg = LogisticRegression(n_jobs = -1)

# Pipeline обучения модели
model_log_reg = Pipeline([('transformer', col_transformer_ohe), ('classifier', log_reg)])

# подбираемые гиперпараметры
parameters = {
    'classifier__C': [0.0001, 0.001, 0.01, 10, 0.0005],
    'classifier__solver': ['saga', 'sag', 'newton-cg']
}

# параметры для обучения с подбором гиперпараметров
grid_cv_lr = GridSearchCV(model_log_reg, parameters, scoring = scorer, cv = cv, n_jobs = -1, refit = 'accuracy')

In [ ]:
# обучение и подбор оптимальных параметров
grid_cv_lr.fit(X_train, y_train)

In [ ]:
# сохранение результатов обучения в соответстующие переменные
result = grid_cv_lr.cv_results_ 
best_params_lr = grid_cv_lr.best_params_
precision_lr = result['mean_test_precision'][np.argmin(result['rank_test_precision'])]
recall_lr = result['mean_test_recall'][np.argmin(result['rank_test_recall'])]
accuracy_lr = grid_cv_lr.best_score_

# вывод результатов обучения модели
print(f'''Оптимальные гиперпараметры LogisticRegression:
{best_params_lr}
Accuracy: {round(accuracy_lr, 2)}
Precision: {round(precision_lr, 2)}
Recall: {round(recall_lr, 2)}''')

**Вывод**

1. Выполнено преобразование входных данных для моделей:
    * Категориальные признаки - техникой OneHotEncoder;
    * Количественные признаки - техникой стандартизации данных StandardSCaler.
2. Инициализирована переменная **scorer** для хранения списка метрик качества оценки модели;
3. Инициализирован объект кросс-валидации с указанием параметров - **cv**:
4. Построен пайплайн преобразования данных и обучения модели логистической регрессии - инициализирована переменная **model_log_reg**;
5. Выполнено обучение модели с перебором значений гиперпараметров;
6. Выведены на экран результаты обучения модели логистической регрессии:
    * Accuracy - 0.63;
    * Precision - 0.75;
    * Recall - 0.43.

#### CatBoost <a class = 'anchor' id = 'Моделирование_baseline_catboost'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Инициализация объекта кросс-валидации для модели CatBoost;
2. Инициализация пользовательской функции:
    * с построением модели CatBoostClassifier;
    * разбиением набора данных на части для обучения техникой кросс-валидации;
    * построения прогноза значений;
    * проверки качества модели на тестовых данных.
3. Инициализация объекта optuna для подбора лучших параметров и оптимизации функции ошибки;
4. Вывод результатов обучения baseline-модели CatBoostClassifier;
5. Инициализация пользовательской функции обучения модели CatBoostClassifier и подбора лучших значений гиперпараметров с библиотекой optuna;
6. Проверка качества модели на тестовых данных;
7. Формирование вывода по итогам данного этапа.

In [ ]:
# инициализация объекта кросс-валидации для модели CatBoost
cv_kfold = KFold(n_splits = SPLITS, random_state = RANDOM_STATE, shuffle = True)

# инициализация пользовательской функции обучения сети с автоматизированным перебором значений гиперпараметров
def objective(trial):
    model = CatBoostClassifier(
        iterations = 1000,
        learning_rate = trial.suggest_float("learning_rate", 1e-3, 1e-1, log = True),
        depth = trial.suggest_int("depth", 4, 10),
        l2_leaf_reg = trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log = True),
        bootstrap_type = trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli"]),
        random_state = RANDOM_STATE,
        verbose = False
    )
    
    scores = 0
    for train_fold_index, val_fold_index in cv_kfold.split(X_train):
        features_train_fold, target_train_fold = X_train.iloc[train_fold_index], y_train.iloc[train_fold_index]
        features_val_fold, target_val_fold = X_train.iloc[val_fold_index], y_train.iloc[val_fold_index]
        
        # train pool
        train = Pool(data = features_train_fold,
                     label = target_train_fold,
                     cat_features = categorical_columns
                     )
        
        # validation pool
        valid = Pool(data = features_val_fold,
                     cat_features = categorical_columns
                     )
        
        # обучение модели с текущими гиперпараметрами
        model.fit(train)
        
        # построение прогноза
        pred = model.predict(valid)
        score = accuracy_score(target_val_fold, pred)
        scores += score
        
    return scores/SPLITS

In [ ]:
# инициализация объекта optuna для подбора лучших параметров и оптимизации функции ошибки
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(study_name = "catboost", direction = "maximize")
study.optimize(objective, n_trials = 50)

In [ ]:
# вывод результатов обучения модели CatBoostClassifier
print("Результаты подборы параметров:")
trial = study.best_trial
print("  Accuracy: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

**Обучение модели CatBoost и вывод на экран лучших значений гиперпараметров**

In [ ]:
# инициализация переменной для хранения модели с лучшими параметрами
model_cb = CatBoostClassifier(**trial.params, verbose = False, random_state = RANDOM_STATE)

In [ ]:
accuracy_cb = 0
precision_cb = 0
recall_cb = 0

# цикл обучения модели с перебором значений параметров
for train_fold_index, val_fold_index in cv_kfold.split(X_train):
    features_train_fold, target_train_fold = X_train.iloc[train_fold_index], y_train.iloc[train_fold_index]
    features_val_fold, target_val_fold = X_train.iloc[val_fold_index], y_train.iloc[val_fold_index]
        
    # train pool
    train = Pool(data = features_train_fold,
                 label = target_train_fold,
                 cat_features = categorical_columns
                 )
    # validation pool
    valid = Pool(data = features_val_fold,
                 cat_features = categorical_columns
                )
    # обучение модели
    model_cb.fit(train)
    # построение прогноза
    pred = model_cb.predict(valid)
    
    accuracy_cb += accuracy_score(target_val_fold, pred)
    precision_cb += precision_score(target_val_fold, pred)
    recall_cb += recall_score(target_val_fold, pred, average = 'binary')
    
# запись финальных метрик
accuracy_cb /= SPLITS
precision_cb /= SPLITS
recall_cb /= SPLITS

# вывод результатов
print(f'''Метрики CatBoost:
Accuracy: {round(accuracy_cb, 2)}
Precision: {round(precision_cb, 2)}
Recall: {round(recall_cb, 2)}''')

In [ ]:
# обучение модели на всех тренировочных данных

# train pool
train = Pool(data = X_train,
             label = y_train,
             cat_features = categorical_columns
             )

# итоговая модель CatBoost
model_cb.fit(train)

**Вывод**

1. Проведена инициализация объекта кросс-валидации для модели CatBoost;
2. Выполнена инициализация пользовательской функции:
    * с построением модели CatBoostClassifier;
    * разбиением набора данных на части для обучения техникой кросс-валидации;
    * построения прогноза значений;
    * проверки качества модели на тестовых данных.
3. Выполнена инициализация объекта optuna для подбора лучших параметров и оптимизации функции ошибки;
4. Выведены результаты обучения baseline-модели CatBoostClassifier:
    * learning_rate - 0.02;
    * depth: 5;
    * l2_leaf_reg: 99.08;
    * bootstrap_type: Bernoulli.
5. Выполнена инициализация пользовательской функции обучения модели CatBoostClassifier и подбора лучших значений гиперпараметров с библиотекой optuna;
6. Выполнена проверка качества модели на тестовых данных;
7. Выведены на экран результаты обучения модели CatBoostClassifier:
    * Accuracy - 0.65;
    * Precision - 0.69;
    * Recall - 0.49.

#### Модель нейронной сети PyTorch <a class = 'anchor' id = 'Моделирование_baseline_pytorch'></a>

**Подготовка данных**

Данный блок характеризуется следующими последовательными действиями:

1. Преобразование входных данных для обучения и тестирования модели нейронной сети - кодирование качественных показателей. Инициализация переменных `X_train_transform` и `X_test_transform`;
2. Преобразование областей признаков в массивы NumPy - инициализация переменных `X_train_array` и `X_test_array`;
3. Преобразование матриц в тензоры PyTorch. Проверка размерности полученных тензоров;
4. Инициализация класса для создания пакетов обучения нейронной сети;
5. Формирование вывода по итогам данного этапа.

In [ ]:
# маштабирование входных данных
X_train_transform = col_transformer_ohe.fit_transform(X_train)
X_test_transform = col_transformer_ohe.transform(X_test)

# преобразование разреженной матрицы в массив NumPy
X_train_array = X_train_transform.toarray()
X_test_array = X_test_transform.toarray()

# преобразование матриц в тензоры
X_train_tensor = torch.from_numpy(X_train_array).to(torch.float32)
X_test_tensor = torch.from_numpy(X_test_array).to(torch.float32)
y_train_tensor = torch.Tensor(y_train.values)
y_test_tensor = torch.Tensor(y_test.values)

# проверка размерности и типа данных тензоров
for x in ['X_train_tensor', 'X_test_tensor', 'y_train_tensor', 'y_test_tensor']:
    print(f'Размерность {x}: {globals()[x].shape} - {globals()[x].dtype}')

In [ ]:
# класс для создания пакетов обучения
class Batch(Dataset):
  def __init__(self, data, labels):
    self.labels = labels
    self.data = data

  def __len__(self):
    return len(self.labels)

  def __getitem__(self, idx):
    label = self.labels[idx]
    data = self.data[idx]
    sample = {'data': data, 'at_fault': label}
    return sample

In [ ]:
# DataLoader выборки
dataset_train = Batch(X_train_tensor, y_train_tensor)
dataset_test = Batch(X_test_tensor, y_test_tensor)

**Вывод по промежуточному этапу**

1. Проведено преобразование входных данных для обучения и тестирования модели нейронной сети. Инициализированы переменные `X_train_transform` и `X_test_transform`;
2. Проведено преобразование областей признаков в массивы NumPy -ь инициализированы переменные `X_train_array` и `X_test_array`;
3. Проведено преобразование матриц в тензоры PyTorch. Размерност полученных тензоров:
    * X_train - (69_441, 26);
    * X_test - (23_148, 26);
    * y_train - (69_441, 1);
    * y_test - (23_148, 1);
4. Проведена инициализация класса для создания пакетов обучения нейронной сети - **Batch**.

**Проектирование архитектуры нейронной сети**

1 Входной слой:

   - Размерность входных данных определяется переменной input_size и равен количеству признаков.
   - Этот слой представлен в виде линейного слоя (nn.Linear), который принимает входные данные и производит линейную трансформацию с помощью весов и смещения.
   
2 Скрытые слои:

   - В данной архитектуре нейронной сети используем четыре скрытых слоя.
   - Каждый скрытый слой состоит из линейного слоя (nn.Linear), за которым следует функция активации.
   - Функции активации, используемые в этой архитектуре, определяются переменными activation_1, activation_2, activation_3, activation_4.
   - Добавление техники регуляризации Dropout, поможет улучшить обобщающую способность нейронной сети и снизит риск переобучения, вероятности выключения нейронов в слоях определяются переменными drop_1, drop_2, drop_3, drop_4. В baseline-модели данную технику применим только к первому слою, затем подберем оптимальные параметры для этого и остальных слоев.
   - Размеры скрытого слоя определяются переменными hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4.
   
3 Выходной слой:

   - Выходной слой представлен линейным слоем (nn.Linear), который преобразует выходные значения скрытых слоев в одно значение.
   - В данном случае, поскольку решается задача бинарной классификации, выходной слой имеет размерность 1 и задается переменной output_size.
   
4 Инициализация весов:
   - Инициализацию весов будем производить с помощью метода Kaiming, что может помочь ускорить процесс обучения и предотвратить затухания/взрыва градиента.

In [ ]:
# определение архитектуры нейронной сети
class Baseline(nn.Module):
    def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, output_size,
                 drop_1, drop_2, drop_3, drop_4,
                 activation_1, activation_2, activation_3, activation_4):
        super(Baseline, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size_1)
        self.act1 = activation_1
        self.drop1 = nn.Dropout(drop_1)
        
        self.fc2 = nn.Linear(hidden_size_1, hidden_size_2)
        self.act2 = activation_2
        self.drop2 = nn.Dropout(drop_2)
        
        self.fc3 = nn.Linear(hidden_size_2, hidden_size_3)
        self.act3 = activation_3
        self.drop3 = nn.Dropout(drop_3)
        
        self.fc4 = nn.Linear(hidden_size_3, hidden_size_4)
        self.act4 = activation_4
        self.drop4 = nn.Dropout(drop_4)
        
        self.fc5 = nn.Linear(hidden_size_4, output_size)
        
        self.init_weights()
        
    def forward(self, x):
        x = self.drop1(self.act1(self.fc1(x)))
        x = self.drop2(self.act2(self.fc2(x)))
        x = self.drop3(self.act3(self.fc3(x)))
        x = self.drop4(self.act4(self.fc4(x)))
        x = self.fc5(x)
        return x
    
    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight)
            m.bias.data.fill_(0.01)

**Ранняя остановка**

Чтобы не подбирать коливо эпох и при этом избежать переобучения, воспользуемся ранней остановкой, для этого напишем класс для ее реализации.

In [ ]:
class CustomEarlyStopping():
    """
    Ранняя остановка: остановить тренировку, если функция потерь не улучшается после определенного количества эпох.
    """
    def __init__(self, patience=5, min_delta=0):
        """
        :param patience: сколько эпох ждать, прежде чем остановиться
        :param min_delta: минимальная разница функций потерь для активации param patience
        """
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif self.best_loss - val_loss > self.min_delta:  # обновление значения функция потерь при улучшении
            self.best_loss = val_loss 
            self.counter = 0
        elif self.best_loss - val_loss < self.min_delta:  # счетчик patience при ухудшении функция потерь
            self.counter += 1
            if self.counter >= self.patience:
                print(f'INFO: Ранняя остановка. Счетчик: {self.counter}/{self.patience}')
                self.early_stop = True  # остановка обучения при достижении максимума счетсика patience

**Baseline-модель нейронной сети**

Данный блок характеризуется следующими последовательными действиями:

1. Ввод переменных для задания констант, формирующих нейронную сеть:
    * Количество скрытых слоев;
    * Количество нейронов;
    * Функции активации на скрытых и выходном слоях;
    * Скорость обучения;
    * Количество эпох обучения;
    * Размер пакета.
2. Создание экземпляра нейронной сети;
3. Инициализация пользовательских функций обучения / тестирования модели;
4. Построение графика прогноз / факт;
5. Обучение нейронной сети и проверка качества прогнозов на тестовой выборке
6. Формирование вывода по итогам данного этапа.

In [ ]:
# параметры Baseline нейронной сети

# размер слоев
input_size = X_train_tensor.shape[1]
hidden_size_1 = 50
hidden_size_2 = 40
hidden_size_3 = 5
hidden_size_4 = 45
output_size = 1

# функции активации
activation_1 = nn.ReLU()
activation_2 = nn.Tanh()
activation_3 = nn.ReLU()
activation_4 = nn.LeakyReLU()

# вероятности выключения нейроны в слое
drop_1 = 0.1
drop_2 = 0.0
drop_3 = 0.0
drop_4 = 0.0

learning_rate = 0.001  # скорость обучения
n_epochs = 1000  # количество эпох обучения
batch_size = 500  # размер пакета

In [ ]:
# экземпляр нейронной сети
model = Baseline(input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, output_size,
                 drop_1, drop_2, drop_3, drop_4,
                 activation_1, activation_2, activation_3, activation_4)
# функция потерь
loss = nn.BCEWithLogitsLoss()

# оптимизатор
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# пакеты (batch)
train_dataloader = DataLoader(dataset_train, batch_size=batch_size, shuffle=True,
                              num_workers=0)
test_dataloader = DataLoader(dataset_test, batch_size=batch_size, num_workers=0)

In [ ]:
def train(model, train_dataloader, test_dataloader, optimizer, loss, n_epochs, patience, min_delta):
    result = {'train_loss': [], 'test_loss': [], 'accuracy': [], 'precision': [], 'recall': [],
              'best_epoch': None, 'stopping_epoch': None, 'best_model': None}

    early_stopping = CustomEarlyStopping(patience, min_delta)

    for epoch in range(n_epochs):
        # обучение
        model.train()
        total_loss_train = 0.0

        for batch in train_dataloader:
            data = batch['data']
            label_train = batch['at_fault']
            label_train = label_train.unsqueeze(1)

            optimizer.zero_grad()

            predictions = model(data)
            loss_value = loss(predictions, label_train.float())  
            total_loss_train += loss_value.item()

            loss_value.backward()
            optimizer.step()

        train_loss = total_loss_train / len(train_dataloader)
        result['train_loss'].append(train_loss)

        # тестирование
        model.eval()
        total_loss_test = 0.0
        total_accuracy_test = 0.0
        total_precision_test = 0.0
        total_recall_test = 0.0
        
        with torch.no_grad():  # отключение вычисления градиентов
            for batch in test_dataloader:
                data = batch['data']
                label_test = batch['at_fault']
                label_test = label_test.unsqueeze(1)

                predictions = model(data)
                loss_value = loss(predictions, label_test.float())
                total_loss_test += loss_value.item()
                
                binary_predictions = (predictions >= 0.5).int()
                total_accuracy_test += accuracy_score(label_test, binary_predictions)
                total_precision_test += precision_score(label_test, binary_predictions, zero_division=1, average='binary')
                total_recall_test += recall_score(label_test, binary_predictions, zero_division=1, average='binary')
                
        test_loss = total_loss_test / len(test_dataloader)
        result['test_loss'].append(test_loss)
        
        result['accuracy'].append(total_accuracy_test / len(test_dataloader))
        result['precision'].append(total_precision_test / len(test_dataloader))
        result['recall'].append(total_recall_test / len(test_dataloader))

        if epoch % 10 == 0 or epoch == n_epochs - 1:
            print(f'Epoch: {epoch} Train Loss: {train_loss:.4f} Test Loss: {test_loss:.4f} Accuracy: {result["accuracy"][-1]:.2f}')

        early_stopping(test_loss)
        if early_stopping.counter == 1:
            result['best_model'] = model
        if early_stopping.early_stop:
            result['best_epoch'] = epoch - patience
            result['stopping_epoch'] = epoch
            print(f"Best epoch: {epoch - patience} Train Loss: {result['train_loss'][-patience]:.4f} Test Loss: {result['test_loss'][-patience]:.4f}")
            print(f"Accuracy: {result['accuracy'][-patience]:.2f} Precision: {result['precision'][-patience]:.2f} Recall: {result['recall'][-patience]:.2f}")
            break
        if epoch == n_epochs - 1:
            result['best_epoch'] = epoch
            result['stopping_epoch'] = epoch
            print(f"Best epoch: {epoch} Train Loss: {result['train_loss'][-1]:.4f} Test Loss: {result['test_loss'][-1]:.4f}")
            print(f"Accuracy: {result['accuracy'][-1]:.2f} Precision: {result['precision'][-1]:.2f} Recall: {result['recall'][-1]:.2f}")

    return result

In [ ]:
# обучение и сохранение результатов
result = train(model=model,
               train_dataloader=train_dataloader,
               test_dataloader=test_dataloader,
               optimizer=optimizer,
               loss=loss,
               n_epochs=n_epochs,
               patience=20,
               min_delta=0)

- Метрики качества Baseline-модель нейронной сети:
    - Accuracy: 0.64
    - Precision: 0.81
    - Recall: 0.30

**График процесса обучения**

In [ ]:
plt.figure(figsize=(8, 4))
plt.plot(result['train_loss'], color='green', label='Train loss')
plt.plot(result['test_loss'], color='blue', label='Test loss')

# отметка Best epoch
plt.axvline(x=result['best_epoch'], color='y', linestyle='--', label='Best epoch')
# отметка Stopping train
plt.axvline(x=result['stopping_epoch'], color='red', linestyle='--', label='Stopping fit')

plt.title('Визуализация процесса обучения:')
plt.xlabel('Epochs')
plt.ylabel('Loss function')
plt.legend(loc='best')
plt.show()

**Вывод**

1. Введены переменные для задания констант, формирующих нейронную сеть:
    * Количество скрытых слоев - 4;
    * Количество нейронов на каждом слое:
        * hidden_size_1 = 50;
        * hidden_size_2 = 40;
        * hidden_size_3 = 5;
        * hidden_size_4 = 45;
        * output_size = 1.
    * Функции активации на скрытых и выходном слоях:
        * activation_1 = nn.ReLU();
        * activation_2 = nn.Tanh();
        * activation_3 = nn.ReLU();
        * activation_4 = nn.LeakyReLU().
    * Скорость обучения - 0.001;
    * Количество эпох обучения - 1_000;
    * Размер пакета - 500.
2. Создан экземпляр нейронной сети - инициализирована переменная **model**;
3. Инициализированы пользовательские функции обучения / тестирования модели;
4. Построен график прогноз / факт;
5. Проведены обучение нейронной сети и проверка качества прогнозов на тестовой выборке:
    * Accuracy: 0.64
    * Precision: 0.81
    * Recall: 0.30.

### Подбор оптимальных гиперпараметров модели прогнозирования <a class = 'anchor' id = 'Моделирование_параметры'></a>

Данный блок характеризуется следующими последовательными действиями:

1. Инициализация пользовательской функции для автоматизированного подбора параметров библиотекой Optuna;
2. Автоматизированный подбор параметров - инициализация переменной **study**;
3. Вывод на экран оптимальных параметров;
4. Тестирование качества модели;
5. Построение графика процесса обучения;
6. Построение сводного результата метрик качества по всем испытуемым моделям;
7. Формирование вывода по итогам данного этапа.

In [ ]:
# подбор оптимальных параметров

y_train_tensor = y_train_tensor.view(-1, 1)
# функция для подбора параметров с помощью Optuna
def objective(trial):
    
    # dropout
    drop_1 = trial.suggest_float('module__drop_1', 0.0, 0.5, step = 0.1)
    drop_2 = trial.suggest_float('module__drop_2', 0.0, 0.5, step = 0.1)
    drop_3 = trial.suggest_float('module__drop_3', 0.0, 0.5, step = 0.1)
    drop_4 = trial.suggest_float('module__drop_4', 0.0, 0.5, step = 0.1)

    # функции активации
    activation_dict = {'nn.LeakyReLU()': nn.LeakyReLU(),
                       'nn.ReLU()': nn.ReLU(),
                       'nn.Tanh()': nn.Tanh(),
                       'nn.ELU()': nn.ELU(),
                       'nn.Sigmoid()': nn.Sigmoid()
                      }

    activation_1 = trial.suggest_categorical('module__activation_1', ['nn.LeakyReLU()', 'nn.ReLU()', 'nn.Tanh()', 'nn.ELU()', 'nn.Sigmoid()'])
    activation_2 = trial.suggest_categorical('module__activation_2', ['nn.LeakyReLU()', 'nn.ReLU()', 'nn.Tanh()', 'nn.ELU()', 'nn.Sigmoid()'])
    activation_3 = trial.suggest_categorical('module__activation_3', ['nn.LeakyReLU()', 'nn.ReLU()', 'nn.Tanh()', 'nn.ELU()', 'nn.Sigmoid()'])
    activation_4 = trial.suggest_categorical('module__activation_4', ['nn.LeakyReLU()', 'nn.ReLU()', 'nn.Tanh()', 'nn.ELU()', 'nn.Sigmoid()'])

    activation_1 = activation_dict.get(activation_1)
    activation_2 = activation_dict.get(activation_2)
    activation_3 = activation_dict.get(activation_3)
    activation_4 = activation_dict.get(activation_4)

    # количество нейронов
    input_size = X_train_tensor.shape[1] 
    output_size = 1

    hidden_size_1 = trial.suggest_int('module__hidden_size_1', 5, 80, step = 5)
    hidden_size_2 = trial.suggest_int('module__hidden_size_2', 5, 80, step = 5)
    hidden_size_3 = trial.suggest_int('module__hidden_size_3', 5, 80, step = 5)
    hidden_size_4 = trial.suggest_int('module__hidden_size_4', 5, 80, step = 5)

    # скорость обучения
    lr = trial.suggest_float('learning_rate', 1e-6, 1e-2, log = True)
    
    # размер пакета обучения
    batch_size = trial.suggest_int('batch_size', 5, 1005, step = 50)
    
    # модель
    model_base = Baseline(input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, output_size,
                          drop_1, drop_2, drop_3, drop_4,
                          activation_1, activation_2, activation_3, activation_4)

    # функция потерь
    loss = nn.BCEWithLogitsLoss() 

    # количество шагов до остановки обучения
    patience = trial.suggest_int('patience', 10, 60, step = 5)

    # параметры подбора параметров модели
    skorch_classifier = NeuralNetClassifier(module = model_base,
                                            device = 'cpu',
                                            verbose = 0,
                                            batch_size = batch_size,
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 500,
                                            lr = lr,
                                            train_split = ValidSplit(cv = 4),
                                            criterion = loss,
                                            callbacks = [
                                                ('val_acc', EpochScoring(scoring = 'accuracy', lower_is_better = False, name = 'val_acc')),
                                                ('estopper', EarlyStopping(lower_is_better = True, monitor = 'val_acc', patience = patience))
                                            ]
                                           )

    # перекрестная проверка
    accuracy = cross_val_score(skorch_classifier, X_train_tensor.numpy().astype('float32'),
                           y_train_tensor.numpy().astype('float32'),
                           n_jobs = -1,
                           cv = 4,
                           scoring = 'accuracy')

    return 1 - accuracy.mean()  # Optuna минимизирует целевую функцию, поэтому используем (1 - accuracy)


In [ ]:
# подбор оптимальных параметров с помощью библиотеки Optuna
study = optuna.create_study(direction = 'minimize')
study.optimize(objective, n_trials = 100)

In [ ]:
# вывод на экран оптимальных параметров
best_params = study.best_trial.params
display('Оптимальные параметры:', best_params)

**Проверка модели с оптимальными параметрами**

In [ ]:
# инициализация оптимальных параметров
activation_dict = {'nn.LeakyReLU()': nn.LeakyReLU(),
                       'nn.ReLU()': nn.ReLU(),
                       'nn.Tanh()': nn.Tanh(),
                       'nn.ELU()': nn.ELU(),
                       'nn.Sigmoid()': nn.Sigmoid()
                      }

input_size = X_train_tensor.shape[1]  
output_size = 1

hidden_size_1 = best_params['module__hidden_size_1']
hidden_size_2 = best_params['module__hidden_size_2']
hidden_size_3 = best_params['module__hidden_size_3']
hidden_size_4 = best_params['module__hidden_size_4']

drop_1 = round(best_params['module__drop_1'], 1)
drop_2 = round(best_params['module__drop_2'], 1)
drop_3 = round(best_params['module__drop_3'], 1)
drop_4 = round(best_params['module__drop_4'], 1)

activation_1 = best_params['module__activation_1']
activation_2 = best_params['module__activation_2']
activation_3 = best_params['module__activation_3']
activation_4 = best_params['module__activation_4']

activation_1 = activation_dict.get(activation_1)
activation_2 = activation_dict.get(activation_2)
activation_3 = activation_dict.get(activation_3)
activation_4 = activation_dict.get(activation_4)

learning_rate = best_params['learning_rate']

batch_size = best_params['batch_size']

patience = best_params['patience']

n_epochs = 1000

In [ ]:
# экземпляр нейронной сети
model = Baseline(input_size, hidden_size_1, hidden_size_2, hidden_size_3, hidden_size_4, output_size,
                      drop_1, drop_2, drop_3, drop_4,
                      activation_1, activation_2, activation_3, activation_4)
# функция потерь
loss = nn.BCEWithLogitsLoss()
# оптимизатор
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
# пакеты (batch)
train_dataloader = DataLoader(dataset_train, batch_size = batch_size, shuffle = True,
                              num_workers = 0)
test_dataloader = DataLoader(dataset_test, batch_size = batch_size, num_workers = 0)

In [ ]:
# обучение и сохранение результатов
result_test = train(model = model,
                    train_dataloader = train_dataloader,
                    test_dataloader = test_dataloader,
                    optimizer = optimizer,
                    loss = loss,
                    n_epochs = n_epochs,
                    patience = patience,
                    min_delta = 0)


**Вывод по промежуточному этапу**

Лучшая модель с оптимальными параметрами:
* Accuracy - 0.63;
* Precision - 0.84;
* Recall - 0.27

In [ ]:
# сохранение метрик качества
accuracy_dl = round(result_test['accuracy'][-patience], 2)
precision_dl = round(result_test['precision'][-patience], 2)
recall_dl = round(result_test['recall'][-patience], 2)

In [ ]:
# построение графика процесса обучения

plt.figure(figsize=(8, 4))
plt.plot(result_test['train_loss'], color='green', label='Train loss')
plt.plot(result_test['test_loss'], color='blue', label='Test loss')

# отметка Best epoch
plt.axvline(x=result_test['best_epoch'], color='y', linestyle='--', label='Best epoch')
# отметка Stopping train
plt.axvline(x=result_test['stopping_epoch'], color='red', linestyle='--', label='Stopping fit')

plt.title('Визуализация процесса обучения:')
plt.xlabel('Epochs')
plt.ylabel('Loss function')
plt.legend(loc='best')
plt.show()

In [ ]:
# построение сводной таблицы показателей моделей

data = {
    'Metrics': ['Accuracy', 'Precision', 'Recall'],
    'LogisticRegression': [round(accuracy_lr, 2), round(precision_lr, 2), round(recall_lr, 2)],
    'CatBoost': [round(accuracy_cb, 2), round(precision_cb, 2), round(recall_cb, 2)],
    'PyTorch_model': [round(accuracy_dl, 2), round(precision_dl, 2), round(recall_dl, 2)]
}
display(pd.DataFrame(data))

**Вывод**

1. Инициализирована пользовательская функция **objective** для автоматизированного подбора параметров;
2. Проведение автоматизированного подбора параметров - инициализирована переменная **study**;
3. Оптимальные параметры модели нейронной сети:
    * module__drop_1: 0.0,
    * module__drop_2: 0.30000000000000004,
    * module__drop_3: 0.4,
    * module__drop_4: 0.2,
    * module__activation_1: 'nn.ELU()',
    * module__activation_2: 'nn.Tanh()',
    * module__activation_3: 'nn.Tanh()',
    * module__activation_4: 'nn.LeakyReLU()',
    * module__hidden_size_1: 25,
    * module__hidden_size_2: 50,
    * module__hidden_size_3: 55,
    * module__hidden_size_4: 10,
    * learning_rate: 0.001043914304211808,
    * batch_size: 855,
    * patience: 50.
4. Проведено тестирование качества модели. Лучшая модель с оптимальными параметрами:
    * Accuracy - 0.63;
    * Precision - 0.84;
    * Recall - 0.27.
5. Построен график процесса обучения;
6. Построен сводный результат метрик качества по всем испытуемым моделям:
    
|Metrics|LogisticRegression|CatBoost|PyTorch_model|
|-------|------------------|--------|-------------|
|Accuracy|0.64|0.65|0.63|
|Precision|0.75|0.69|0.84|
|Recall|0.43|0.49|0.27|

Все модели обладают **не высокой способностью прогнозировать целевую переменную** - факт возникновения ДТП.

Основной целью использования модели является **минимизация ложноотрицательных ответов модели** (т.е., прогноз клиентов, которые стали причиной ДТП, но были определены моделью как 0).

По совокупному анализу значений метрик лучшей моделью является CatBoost:
* Лучший результат Accuracy - 0.65 - указывает на наилучшую обобщающую спосовность модели;
* Высокий Recall:  Recall важен в данном контексте, поскольку он показывает, как много положительных объектов (виновников ДТП) модель обнаружила из всех положительных объектов в данных. CatBoost показывает Recall 0.47, что означает, что модель правильно определяет около 47% всех виновников ДТП.
* Относительно высокая точность: Несмотря на то, что Recall в CatBoost ниже, чем в PyTorch_model, его точность (Precision) также достаточно высока (0.69). Precision показывает, как много из всех объектов, которые модель предсказала как положительные, действительно являются положительными. Это важно, чтобы минимизировать ложноположительные ответы.

### Проверка качества модели на тестовых данных <a class = 'anchor' id = 'Моделирование_качество'></a>

In [ ]:
# train pool
test = Pool(data=X_test,
            cat_features=categorical_columns
            )
    
# предсказание
pred = model_cb.predict(test)
    
accuracy = accuracy_score(y_test, pred)
precision = precision_score(y_test, pred)
recall = recall_score(y_test, pred, average='binary')

# вывод результатов
print(f'''Метрики CatBoost на тестовых данных:
Accuracy: {round(accuracy, 2)}
Precision: {round(precision, 2)}
Recall: {round(recall, 2)}''')

- Метрики качества модели на тестовых данных подтверждают соответствие метрик, полученных на данных для обучения, что говорит о том, что модель адекватна и отсутствует переобучение.
- Accuracy (Точность) составляет 0.66, что означает, что модель правильно классифицировала 66% тестовых данных.
- Precision (Точность) составляет 0.7, что говорит о том, что из всех объектов, которые модель классифицировала как виновников ДТП, 70% действительно являются таковыми.
- Recall (Полнота) равен 0.49, что означает, что модель обнаруживает 49% всех действительных виновников ДТП из всех положительных случаев.

__Визуализация данных тестирования__

In [ ]:
# получение матрицы ошибок
cm = confusion_matrix(y_test, pred)

In [ ]:
# визуализация матрицы ошибок
plt.figure(figsize=(3, 3))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Предсказанные значения")
plt.ylabel("Истинные значения")
plt.title("Матрица ошибок")
plt.show()

In [ ]:
# визуализация Recall и Precision
plt.figure(figsize=(4, 3))
plt.bar(['Полнота (Recall)', 'Точность (Precision)'], [recall, precision], color=['teal', 'seagreen'])
plt.ylim(0, 1)
plt.ylabel('Значение')
plt.title('Полнота и точность')
plt.show()

- Модель имеет неплохую точность (Precision) в предсказании виновников ДТП (при условии, что она предсказывает "1"), что может быть полезным, чтобы уменьшить количество ложно положительных предсказаний.
- Однако, модель имеет относительно низкий Recall, что означает, что она не слишком хорошо обнаруживает реальных виновников ДТП (число ложно отрицательных предсказаний велико).

### Анализ важности основных факторов <a class = 'anchor' id = 'Моделирование_факторы'></a>

Для анализа важности признаков и визуализации данных используем библиотеку SHAP (SHapley Additive exPlanations) 

In [ ]:
# инициализация Explainer
explainer = shap.Explainer(model_cb)

# получение значений SHAP для тестовых данных
shap_values = explainer.shap_values(X_test)

# визуализация SHAP значений
shap.summary_plot(shap_values, X_test, plot_type='bar', show=False, color='teal')
plt.title('Важность признаков (SHAP)')
plt.show()

Как видно из графика, самым главным признаком, влияющий на фак аварии является `party_sobriety` - трезвость участника, при предобработки данных мы разделили записи в зависимости от степени опьянения. Посмотрим зависимость вероятности стать виновником ДТП от уровня опьянения:

In [ ]:
# сводная таблица для среднего значения 'at_fault' по уровню опьянения 'party_sobriety'
mean_at_fault = df.groupby('party_sobriety')['at_fault'].mean()

# построение графика
sns.set_style("whitegrid")
plt.figure(figsize=(8, 4))
sns.lineplot(x=mean_at_fault.index, y=mean_at_fault.values, marker='o', color='b')
plt.xlabel('Уровень опьянения (party_sobriety)')
plt.ylabel('Среднее значение at_fault')
plt.title('Зависимость вероятности стать виновником ДТП от уровня опьянения')
plt.xticks(mean_at_fault.index)
plt.show()

**Вывод**

Из графика видно, что с увеличением уровня опьянения (от 0 до 3) вероятность стать виновником ДТП увеличивается. Чем выше уровень опьянения, тем больше вероятность попадания в аварию, где водитель считается виновным.

Чтобы учесть этот фактор во время посадки водителя, можно использовать различные технические средства и системы, такие как:

- Алкотестеры: Установка алкотестеров в автомобиле позволит водителю проверить свой уровень опьянения перед поездкой и принять решение о том, стоит ли ему садиться за руль.

- Автоматические системы блокировки двигателя: Эти системы могут быть настроены на блокировку двигателя, если уровень опьянения водителя превышает определенный уровень.

- Интеллектуальные сенсоры и системы мониторинга: Можно разработать системы, которые будут контролировать поведение водителя и выдавать предупреждения или автоматически предпринимать меры, если обнаружится признак опьянения.

- Системы предупреждения о состоянии водителя: Использование камер и сенсоров для мониторинга поведения водителя и выявления признаков усталости или опьянения.

Эти технологии и системы могут помочь снизить вероятность аварий, связанных с опьянением водителей, и повысить безопасность дорожного движения. Однако следует помнить, что самое эффективное средство предотвращения аварий - это не садиться за руль в состоянии опьянения.

## Общий вывод <a class = 'anchor' id = 'Вывод'></a>

1. Выполнена установка необходимых библиотек Python:
    * phik - для построения матрицы корреляции;
    * optuna - поиск гиперпараметров для модели;
    * skorch - для возможности использования PyTorch совместно с Sklearn;
    * shap - для построения визуализации важности признаков;
    * выполнено обновление библиотеки scikit-learn.
2. Импортированы библиотеки Python:
    * для подключения к базе данных:
        * psycopg2;
        * sqlalchemy;
        * sql, extras.
    * для манипулирования данными:
        * pandas;
        * numpy.
    * для визуализации данных:
        * matplotlib.pyplot;
        * seaborn.
    * для проведения математических операций:
        * math
    * для работы с датой и временем:
        * datetime
    * для вычисления корреляции данных:
        * phik_matrix
    * для решения задач машинного обучения:
        * механизмы построения нейронной сети:
            * torch.nn.;
            * optim;
            * Dataset, DataLoader;
            * NeuralNetClassifier;
            * ValidSplit;
            * EpochScoring, EarlyStopping
        * метрики оценки эффективности моделей классификации;
        * train_test_split - механизм разделения данных;
        * StandardScaler - стандартизация данных;
        * OneHotEncoder - one-hot кодирование категориальных признаков;
        * OrdinalEncoder - кодирование значений ранговых признаков;
        * линейные модели классификации;
        * библиотека CatBoost;
        * библиотека shap для анализа важности признаков.
3. Инициализированы переменные **RANDOM_STATE**, **TEST_SIZE** и **SPLITS** для фиксирования "случайности", размера тестовой выборки и количества сплитов на этап кросс-валидации соответственно;
4. Инициализированы параметры подключения к БД.
5. Выполнено подключение к Базе Данных по установленным параметрам входа;
6. Проинициализирована пользовательская функция выполнения запросов - **request**. Выполнен тестовый запрос;
7. Инициализирован объект **engine** для подключения к БД;
8. Выведена на экран первичная информация о наборах данных:
    * `collisions`:
        * Общее количество записей - 1_400_000;
        * Общее количество признаков - 20;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `case_ids`:
        * Общее количество записей - 1_400_400;
        * Общее количество признаков - 2;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `parties`:
        * Общее количество записей - 2_752_408;
        * Общее количество признаков - 9;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
    * `vehicles`:
        * Общее количество записей - 1_021_234;
        * Общее количество признаков - 6;
        * Первичный ключ отсутствует;
        * snake_case в названиях столбцов.
9. Проведен первичный анализ наборов данных:
    * Изучен временной диапазон располагаемых данных - **с 01.01.2009 по 26.07.2020**;
    * Построены сводные таблицы и визуализации для изучения взаимосвязей количества аварий и:
        * месяцев года:
            * Топ-3 месяца по количеству аварий:
                * Март;
                * Январь;
                * Май.
        * частых причин нарушений:
            * Превышение скорости (speeding) - 34.5% от общего числа;
            * Неправильный поворот (improper turning) - 18.8%;
            * Приоритет движения автомобилей (automobile right of way) - 12.5%;
            * Езда в нетрезвом виде (dui) - 8.7%;
            * Опасное пересечение полосы движения (unsafe lane change) - 7.4%.
        * погодных условий:
            * Ясно (clear) - 80.07% от общего числа;
            * Облачно (cloudy) - 14.5%;
            * Дождь (raining) - 4.1%;
            * Без указания погоды (None) - 0.5%;
            * Туман (fog) - 0.4%.
        * возраста транспортных средств / вида нарушения:
            * Независимо от возрастной категории автомобилей, самыми частыми типами аварий являются:
            * "rear end" (столкновение сзади);
            * "broadside" (боковое столкновение).
            * Типы аварий остаются одинаковыми для двух возрастных категорий автомобилей (исключение составляют только типы sideswipe и hit object, которые "поменялись местами" для новых автомобилей)
        * времени нарушения / состояния дороги:
            * Наибольшее количество ДТП происходит на **сухой дороге**. Можно провести связь между подобными условиями для езды и одной из самых частых причин аварии - **превышение скорости**. Сухая дорога обеспечивает лучший контакт шин с дорогой, и невольно побуждает водителей ехать быстрее разрешенного ограничения скорости.
            * С другой стороны, наличие дождя, снега или скользкого дорожного покрытия создает более опасные условия для движения, что заставляет водителей быть более осторожными, снизить скорость и быть более внимательными к дорожной обстановке или совсем отказаться от использования автомобиля. Это может приводить к снижению количества ДТП в условиях плохой погоды.
            * Вне зависимости от состояния дороги (сухой, мокрый, снежный или скользкий), **большинство ДТП происходят утром и днем**. Это могут быть утренние рабочие часы, а также общий дневной трафик в рабочие и выходные дни
            * В случае, когда **дорога заснежена** (снежное состояние), **наибольшее количество ДТП происходит именно утром**. Этому может быть объяснение в виде неочищенной дороги от наледи и снега.
10. Произведен первичный отбор данных для построения модели по условиям:
    * Виновник аварии - **car** (автомобиль);
    * Тип повреждений - значительные повреждения любого из участников (исключая **scratch** (царапина));
    * Временные рамки для моделирования - **2012 год**.
11. Произведен отбор признаков для моделирования с исключением следующих показателей:
    * Показатели идентификации:
        * **id** - номер записи в БД;
        * **case_id** - идентификатор ДТП;
        * **party_number** - номер участника происшествия;
        * **county_city_location** - идентификатор географического района, где произошло ДТП.
    * Показатели пост-анализа ДТП:
        * **party_count** - Количество участников
        * **primary_collision_factor** - Основной фактор аварии
        * **pcf_violation_category** - Категория нарушения
        * **type_of_collision** - Тип аварии
        * **motor_vehicle_involved_with** - Дополнительные участники ДТП
        * **intersection** - Является ли место происшествие перекрёстком
        * **collision_damage** - Серьёзность происшествия 
12. Сформированный набор подготовлен для последующей предобработки данных и исследовательского анализа.
13. Выведена общая информация о наборе `df` на экран:
    * В наборе присутствуют пустые значения по нескольким признакам. Стратегия их заполнения будет выбрана на этапе исследовательского анализа данных;
    * Типы данных **соответствуют** сущностям значений по всем рассматриваемым признакам;
14. Выполнена проверка датасетов на явные дубликаты - **Дубликаты были удалены из набора**;
15. Построена Phik-матрица корреляции - произведен анализ взаимосвязей **количественных и качественных** признаков с целевой переменной.
    * Заметна линейная взаимосвязь между целевой переменной и указанными признаками:
        * **insurance_premium**;
        * **party_sobriety**;
        * **party_drug_physical**;
        * **vehicle_transmission**;
        * **county_location**;
        * **road_surface**;
        * **control_device**
    * Слабую корреляцию с целевой переменной демонстрируют признаки:
        * **cellphone_in_use**;
        * **vehicle_type**;
        * **vehicle_age**;
        * **distance**;
        * **direction**;
        * **weather_1**;
        * **location_type**;
        * **road_condition_1**;
        * **lighting**.
    * Наблюдается заметная корреляция между признаками:
        * **party_sobriety - party_drug_physical**. Сильная взаимосвязь наблюдается по причине семантики самих признаков: Водитель под алкоголем является не трезвым, что дополнительно фиксируется в признаке party_drug_physical;
        * **road_surface - weather_1**. Состояние дорожного покрытия зависит от погодных условий.
16. Построена матрица корреляции Спирмана - произведен анализ взаимосвязей количественных признаков с целевой переменной:
    * **insurance_premium**. Чем выше страховка, тем меньше вероятность возникновения ДТП. Такая связь объясняется дороговизной самого автомобиля с сопутствующей на него суммой страховки;
    * **vehicle_age**. Чем старше автомобиль, тем меньше вероятность возникновения ДТП. Возможно, что это объясняется тем, что водители подобных транспортных средств обладают большим опытом вождения.

    * Для этапа построения моделей машинного обучения будут отброшены следующие показатели, но после проведения исследовательского анализа данных и разбора семантики значений:
        * **cellphone_in_use**;
        * **vehicle_type**;
        * **distance**;
        * **direction**;
        * **location_type**;
        * **road_condition**
17. Произведен отбор категориальных и количественных значений в отдельные переменные - **categorial_columns** и **numeric_columns**;
18. Подготовлены наборы данных для обучения и тестирования моделей:
    * Разделение исходного набора `df` на область признаков и вектор целевой переменной - инициализация переменных `X` и **y** соответственно;
    * Инициализация переменных `X_train`, `X_test`, **y_train** и **y_test** для хранения обучающих и тестовых наборов данных;
19. Проведена проверка баланса классов в наборах данных. Баланс классов в выборках сохранен.
20. Выполнено преобразование входных данных для моделей:
    * Категориальные признаки - техникой OneHotEncoder;
    * Количественные признаки - техникой стандартизации данных StandardSCaler.
21. Инициализирована переменная **scorer** для хранения списка метрик качества оценки модели;
22. Инициализирован объект кросс-валидации с указанием параметров - **cv**:
23. Построен пайплайн преобразования данных и обучения модели логистической регрессии - инициализирована переменная **model_log_reg**;
24. Выполнено обучение модели с перебором значений гиперпараметров;
25. Выведены на экран результаты обучения модели логистической регрессии:
    * Accuracy - 0.63;
    * Precision - 0.75;
    * Recall - 0.43.
26. Проведена инициализация объекта кросс-валидации для модели CatBoost;
27. Выполнена инициализация пользовательской функции:
    * с построением модели CatBoostClassifier;
    * разбиением набора данных на части для обучения техникой кросс-валидации;
    * построения прогноза значений;
    * проверки качества модели на тестовых данных.
28. Выполнена инициализация объекта optuna для подбора лучших параметров и оптимизации функции ошибки;
29. Выведены результаты обучения baseline-модели CatBoostClassifier:
    * learning_rate - 0.02;
    * depth: 5;
    * l2_leaf_reg: 99.08;
    * bootstrap_type: Bernoulli.
30. Выполнена инициализация пользовательской функции обучения модели CatBoostClassifier и подбора лучших значений гиперпараметров с библиотекой optuna;
31. Выполнена проверка качества модели на тестовых данных;
32. Выведены на экран результаты обучения модели CatBoostClassifier:
    * Accuracy - 0.65;
    * Precision - 0.69;
    * Recall - 0.49.
33. Проведено преобразование входных данных для обучения и тестирования модели нейронной сети. Инициализированы переменные `X_train_transform` и `X_test_transform`;
34. Проведено преобразование областей признаков в массивы NumPy -ь инициализированы переменные `X_train_array` и `X_test_array`;
35. Проведено преобразование матриц в тензоры PyTorch. Размерност полученных тензоров:
    * X_train - (69_441, 26);
    * X_test - (23_148, 26);
    * y_train - (69_441, 1);
    * y_test - (23_148, 1);
36. Проведена инициализация класса для создания пакетов обучения нейронной сети - **Batch**.
37. Введены переменные для задания констант, формирующих нейронную сеть:
    * Количество скрытых слоев - 4;
    * Количество нейронов на каждом слое:
        * hidden_size_1 = 50;
        * hidden_size_2 = 40;
        * hidden_size_3 = 5;
        * hidden_size_4 = 45;
        * output_size = 1.
    * Функции активации на скрытых и выходном слоях:
        * activation_1 = nn.ReLU();
        * activation_2 = nn.Tanh();
        * activation_3 = nn.ReLU();
        * activation_4 = nn.LeakyReLU().
    * Скорость обучения - 0.001;
    * Количество эпох обучения - 1_000;
    * Размер пакета - 500.
38. Создан экземпляр нейронной сети - инициализирована переменная **model**;
39. Инициализированы пользовательские функции обучения / тестирования модели;
40. Построен график прогноз / факт;
41. Проведены обучение нейронной сети и проверка качества прогнозов на тестовой выборке:
    * Accuracy: 0.64
    * Precision: 0.81
    * Recall: 0.30.
42. Инициализирована пользовательская функция **objective** для автоматизированного подбора параметров;
43. Проведение автоматизированного подбора параметров - инициализирована переменная **study**;
44. Оптимальные параметры модели нейронной сети:
    * module__drop_1: 0.0,
    * module__drop_2: 0.30000000000000004,
    * module__drop_3: 0.4,
    * module__drop_4: 0.2,
    * module__activation_1: 'nn.ELU()',
    * module__activation_2: 'nn.Tanh()',
    * module__activation_3: 'nn.Tanh()',
    * module__activation_4: 'nn.LeakyReLU()',
    * module__hidden_size_1: 25,
    * module__hidden_size_2: 50,
    * module__hidden_size_3: 55,
    * module__hidden_size_4: 10,
    * learning_rate: 0.001043914304211808,
    * batch_size: 855,
    * patience: 50.
45. Проведено тестирование качества модели. Лучшая модель с оптимальными параметрами:
    * Accuracy - 0.63;
    * Precision - 0.84;
    * Recall - 0.27.
46. Построен график процесса обучения;
47. Построен сводный результат метрик качества по всем испытуемым моделям:
    
|Metrics|LogisticRegression|CatBoost|PyTorch_model|
|-------|------------------|--------|-------------|
|Accuracy|0.64|0.65|0.63|
|Precision|0.75|0.69|0.84|
|Recall|0.43|0.49|0.27|

Все модели обладают **не высокой способностью прогнозировать целевую переменную** - факт возникновения ДТП.

Основной целью использования модели является **минимизация ложноотрицательных ответов модели** (т.е., прогноз клиентов, которые стали причиной ДТП, но были определены моделью как 0).

48. По совокупному анализу значений метрик лучшей моделью является CatBoost:
    * Лучший результат Accuracy - 0.65 - указывает на наилучшую обобщающую спосовность модели;
    * Высокий Recall:  Recall важен в данном контексте, поскольку он показывает, как много положительных объектов (виновников ДТП) модель обнаружила из всех положительных объектов в данных. CatBoost показывает Recall 0.47, что означает, что модель правильно определяет около 47% всех виновников ДТП.
    * Относительно высокая точность: Несмотря на то, что Recall в CatBoost ниже, чем в PyTorch_model, его точность (Precision) также достаточно высока (0.69). Precision показывает, как много из всех объектов, которые модель предсказала как положительные, действительно являются положительными. Это важно, чтобы минимизировать ложноположительные ответы.
49. Построен график анализа важности признаков. Из графика видно, что с увеличением уровня опьянения (от 0 до 3) вероятность стать виновником ДТП увеличивается. Чем выше уровень опьянения, тем больше вероятность попадания в аварию, где водитель считается виновным.



Чтобы учесть этот фактор во время посадки водителя, можно использовать различные технические средства и системы, такие как:

* Алкотестеры: Установка алкотестеров в автомобиле позволит водителю проверить свой уровень опьянения перед поездкой и принять решение о том, стоит ли ему садиться за руль.

* Автоматические системы блокировки двигателя: Эти системы могут быть настроены на блокировку двигателя, если уровень опьянения водителя превышает определенный уровень.

* Интеллектуальные сенсоры и системы мониторинга: Можно разработать системы, которые будут контролировать поведение водителя и выдавать предупреждения или автоматически предпринимать меры, если обнаружится признак опьянения.

* Системы предупреждения о состоянии водителя: Использование камер и сенсоров для мониторинга поведения водителя и выявления признаков усталости или опьянения.

**Эти технологии и системы могут помочь снизить вероятность аварий, связанных с опьянением водителей, и повысить безопасность дорожного движения. Однако следует помнить, что самое эффективное средство предотвращения аварий - это не садиться за руль в состоянии опьянения**